In [1]:
"""
@author: pho
"""
## Automatic debugger on exception raised
%pdb off
%load_ext autoreload
%autoreload 2

import sys
import importlib
from pathlib import Path
from copy import deepcopy

from numba import jit
import numpy as np
import pandas as pd

%config Completer.use_jedi = False

# # required to enable non-blocking interaction:
%gui qt5

import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to plot Speed vs. Pf Desnity Metrics
from pyphoplacecellanalysis.External.pyqtgraph.Qt import QtCore, QtGui
import pyphoplacecellanalysis.External.pyqtgraph.opengl as gl # for 3D raster plot

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences()
# np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%g" % x))
# np.set_printoptions(edgeitems=None, linewidth=100000, formatter=None)
set_pho_preferences_concise()

# For Progress bars:
# from tqdm.notebook import tqdm, trange

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import get_arguments_as_optional_dict, inspect_callable_arguments
from pyphocorehelpers.indexing_helpers import partition, build_spanning_bins, compute_spanning_bins, compute_position_grid_size, compute_paginated_grid_config
from pyphocorehelpers.print_helpers import PrettyPrintable, WrappingMessagePrinter, print_keys_if_possible, debug_dump_object_member_shapes
from pyphocorehelpers.DataStructure.dynamic_parameters import DynamicParameters
from pyphocorehelpers.performance_timing_helpers import WrappingPerformanceTimer
from pyphocorehelpers.gui.interaction_helpers import CallbackWrapper
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
# from pyphoplacecellanalysis.General.SessionSelectionAndFiltering import batch_filter_session, build_custom_epochs_filters
from neuropy.core.session.KnownDataSessionTypeProperties import KnownDataSessionTypeProperties

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.ComputationFunctionRegistryHolder import ComputationFunctionRegistryHolder
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DisplayFunctionRegistryHolder import DisplayFunctionRegistryHolder

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.core.session.data_session_loader import DataSessionLoader
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.placefields import PlacefieldComputationParameters

from neuropy.core.laps import Laps  # Used for adding laps in KDiba mode
from neuropy.utils.efficient_interval_search import get_non_overlapping_epochs, drop_overlapping # Used for adding laps in KDiba mode

from neuropy.core.epoch import NamedTimerange

from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## Plotting Helpers:
from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_Matrix import MatrixRenderingWindow
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import pyqtplot_plot_image_array, pyqtplot_plot_image

known_data_session_type_properties_dict = DataSessionFormatRegistryHolder.get_registry_known_data_session_type_dict()
active_data_session_types_registered_classes_dict = DataSessionFormatRegistryHolder.get_registry_data_session_type_class_name_dict()
# DataSessionFormatRegistryHolder.get_registry()

enable_saving_to_disk = False
# common_parent_foldername = Path(r'R:\Dropbox (Personal)\Active\Kamran Diba Lib\Pho-Kamran-Meetings\Final Placemaps 2021-01-14')
common_parent_foldername = Path(r'R:\Dropbox (Personal)\Active\Kamran Diba Lib\Pho-Kamran-Meetings\2022-01-16')

def JupyterLab_excepthook(exc_type, exc_value, exc_tb):
    tb = "".join(traceback.format_exception(exc_type, exc_value, exc_tb))
    print("JupyterLab_excepthook error catched!:")
    print("\t error message:\n", tb)
    QtWidgets.QApplication.quit()
    # or QtWidgets.QApplication.exit(0)
    
sys.excepthook = JupyterLab_excepthook

## Concise Numpy Array Printing:
# np.set_string_function(lambda x:f'np.array[{np.shape(x)}]', repr=True)
# # with np.printoptions(precision=2, edgeitems=2, linewidth=144):
# with np.printoptions(precision=3, edgeitems=2, linewidth=100000):
#     # active_computed_data['pf2D'].ratemap
#     print(active_computed_data['pf2D'])
#     # print(active_computed_data['pf2D'].ratemap)
    
    
# np.set_string_function(lambda x:f'np.array[{np.shape(x)}]', repr=True)
# np.set_string_function(None, repr=True)

from pyphocorehelpers.general_helpers import get_arguments_as_optional_dict, CodeConversion
# cls.compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=
# get_arguments_as_optional_dict('speed_thresh=10.0, grid_bin=cls.compute_position_grid_bin_size(sess.position.x, sess.position.y, num_bins=(64, 64)), smooth=(2.0, 2.0), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None')
# CodeConversion.get_arguments_as_optional_dict(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(2.0, 2.0), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None)
# CodeConversion.get_arguments_as_optional_dict(max_num_spikes_per_neuron=20000, kleinberg_parameters=DynamicContainer(s=2, gamma=0.2), use_progress_bar=False, debug_print=False)




Automatic pdb calling has been turned OFF


In [2]:
## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer, override_dict, overriding_dict_with, get_dict_subset
from neuropy.utils.position_util import compute_position_grid_size

def _test_build_default_computation_configs(sess, **kwargs):
    """ OPTIONALLY can be overriden by implementors to provide specific filter functions """
    kwargs.setdefault('pf_params', PlacefieldComputationParameters(**override_dict({'speed_thresh': 10.0, 'grid_bin': (9.0, 9.0), 'smooth': (2.0, 2.0), 'frate_thresh': 0.2, 'time_bin_size': 1.0, 'computation_epochs': None}, kwargs)))
    kwargs.setdefault('spike_analysis', DynamicContainer(**{'max_num_spikes_per_neuron': 20000,
                                                             'kleinberg_parameters': DynamicContainer(**{'s': 2, 'gamma': 0.2}).override(kwargs),
                                                             'use_progress_bar': False,
                                                             'debug_print': False}).override(kwargs))        
    return [DynamicContainer(pf_params=kwargs['pf_params'], spike_analysis=kwargs['spike_analysis'])]


# _test_build_default_computation_configs(sess)

def build_eloy_computation_configs(sess, **kwargs):
    """ OPTIONALLY can be overriden by implementors to provide specific filter functions """
    # (4.0, 4.0)cm bins, (6.0, 6.0)cm gaussian smoothing
    # peak frate > 2Hz 
    # return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(6.0, 6.0), frate_thresh=0.2, time_bin_size=1.0, computation_epochs = None), spike_analysis=None)]
    return [DynamicContainer(pf_params=PlacefieldComputationParameters(speed_thresh=10.0, grid_bin=(4.0, 4.0), smooth=(2.5, 2.5), frate_thresh=0.8, time_bin_size=10.0, computation_epochs = None), spike_analysis=None)]

## Bapun Format:

In [ ]:
active_data_mode_name = 'bapun'
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]
curr_bapun_pipeline = NeuropyPipeline.init_from_known_data_session_type(active_data_mode_name, active_data_mode_type_properties)
curr_active_pipeline = curr_bapun_pipeline
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
active_session_computation_configs = active_data_mode_registered_class.build_default_computation_configs(sess=curr_active_pipeline.sess)

In [ ]:
## Note that pickling the whole spikes_df dataframe when done is very quick, especially to load!
active_file_suffix = '.spikes_df_full.pkl'
spikes_df_save_path = curr_active_pipeline.sess.filePrefix.with_suffix(active_file_suffix)
print(f'spikes_df_save_path: {spikes_df_save_path}')

In [ ]:
curr_active_pipeline.sess.spikes_df.to_pickle(spikes_df_save_path)  # where to save it, usually as a .pkl

In [ ]:
loaded_spikes_df = pd.read_pickle(spikes_df_save_path)
loaded_spikes_df

In [ ]:
curr_active_pipeline.filter_sessions(active_session_filter_configurations)
curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_blacklist=['_perform_spike_burst_detection_computation', '_perform_velocity_vs_pf_density_computation', '_perform_velocity_vs_pf_simplified_count_density_computation']) # SpikeAnalysisComputations._perform_spike_burst_detection_computation
## _perform_velocity_vs_pf_density_computation: causes AssertionError: After AOC normalization the sum over each cell should be 1.0, but it is not! [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan] with 1D placefields!
curr_active_pipeline.prepare_for_display(should_smooth_maze=False) # TODO: pass a display config

## Rachel Format:

In [ ]:
active_data_mode_name = 'rachel'
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]
curr_rachel_pipeline = NeuropyPipeline.init_from_known_data_session_type(active_data_mode_name, active_data_mode_type_properties)
curr_active_pipeline = curr_rachel_pipeline
active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
active_session_computation_configs = active_data_mode_registered_class.build_default_computation_configs(sess=curr_active_pipeline.sess)

In [ ]:
curr_active_pipeline.filter_sessions(active_session_filter_configurations)
curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_blacklist=['_perform_spike_burst_detection_computation']) # Causes "IndexError: index 59 is out of bounds for axis 0 with size 59"
curr_active_pipeline.prepare_for_display(should_smooth_maze=True) # TODO: pass a display config

## KDiba Format:

In [3]:
# %%cache
# from cached_property import cached_property
active_data_mode_name = 'kdiba'
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m
# From pre-computed .mat files:
## 07:
# basedir = r'/run/media/halechr/MoverNew/data/KDIBA/gor01/one/2006-6-07_11-26-53' # Lab Linux Workstation
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'

curr_kdiba_pipeline = NeuropyPipeline.init_from_known_data_session_type(active_data_mode_name, active_data_mode_type_properties, override_basepath=Path(basedir), override_post_load_functions=[])
curr_active_pipeline = curr_kdiba_pipeline
# active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
active_session_filter_configurations = active_data_mode_registered_class.build_filters_pyramidal_epochs(sess=curr_active_pipeline.sess, epoch_name_whitelist=['maze1'])

# active_session_computation_configs = active_data_mode_registered_class.build_default_computation_configs(sess=curr_active_pipeline.sess)
active_session_computation_configs = build_eloy_computation_configs(sess=curr_active_pipeline.sess)
curr_active_pipeline.filter_sessions(active_session_filter_configurations)
# curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_whitelist=['_perform_baseline_placefield_computation', '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'], debug_print=False)
curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_blacklist=['_perform_spike_burst_detection_computation'], debug_print=False) # whitelist: ['_perform_baseline_placefield_computation']
curr_active_pipeline.prepare_for_display(should_smooth_maze=True) # TODO: pass a display config

basedir is already Path object.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikes.mat... done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy... 2006-6-07_11-26-53.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results to spikes_df... done.
	 Saving updated interpolated spike position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.interpolated_spike_positions.n

## Hiro Format:

In [ ]:
active_data_mode_name = 'hiro'
active_data_mode_registered_class = active_data_session_types_registered_classes_dict[active_data_mode_name]
active_data_mode_type_properties = known_data_session_type_properties_dict[active_data_mode_name]
## Data must be pre-processed using the MATLAB script located here: 
# C:\Users\pho\repos\PhoDibaLab_REM_HiddenMarkovModel\DEVELOPMENT\NeuroPyExporting2022\PhoNeuroPyConvert_ExportAllToPython_MAIN.m
# From pre-computed .mat files:
## RoyMaze1: 
basedir = r'R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1'
## RoyMaze2:
# basedir = r'R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze2'
curr_hiro_pipeline = NeuropyPipeline.init_from_known_data_session_type(active_data_mode_name, active_data_mode_type_properties, override_basepath=Path(basedir))
curr_active_pipeline = curr_hiro_pipeline

In [ ]:
curr_active_pipeline.sess.epochs._data

In [ ]:
## TODO: this fixes the epoch's labels after the fact, but it doesn't fix everything. [['pre_sleep'], ['track'], ['post_sleep']]
curr_active_pipeline.sess.epochs._data.label = ['pre_sleep', 'track', 'post_sleep']

# curr_active_pipeline.sess.epochs.labels = ['pre_sleep', 'track', 'post_sleep']

In [ ]:
curr_active_pipeline.sess.epochs['track']

In [ ]:
curr_active_pipeline.sess.neurons.t_start = curr_active_pipeline.sess.epochs['track'][0]

In [ ]:
# active_session_filter_configurations = active_data_mode_registered_class.build_default_filter_functions(sess=curr_active_pipeline.sess) # build_filters_pyramidal_epochs(sess=curr_kdiba_pipeline.sess)
active_session_filter_configurations = active_data_mode_registered_class.build_track_only_filter_functions(sess=curr_active_pipeline.sess)
active_session_computation_configs = active_data_mode_registered_class.build_default_computation_configs(sess=curr_active_pipeline.sess)

In [ ]:
curr_active_pipeline.filter_sessions(active_session_filter_configurations)

In [ ]:
curr_active_pipeline.filtered_session_names
curr_active_pipeline.filtered_epochs
curr_active_pipeline.filtered_sessions['track']

In [ ]:
curr_active_pipeline.perform_computations(active_session_computation_configs[0], computation_functions_name_blacklist=['_perform_spike_burst_detection_computation', '_perform_velocity_vs_pf_density_computation', '_perform_velocity_vs_pf_simplified_count_density_computation'])

In [ ]:
curr_active_pipeline.prepare_for_display(should_smooth_maze=True) # TODO: pass a display config

# Common: Display
Common visualization and display functions for both forms of data/pipelines:

In [5]:
# %%heat
# To view the heatmap
active_config_name = 'maze1'
# active_config_name = 'maze2'
# active_config_name = 'maze'

# Get relevant variables:
# curr_active_pipeline is set above, and usable here
sess: DataSession = curr_active_pipeline.filtered_sessions[active_config_name]

active_computation_results = curr_active_pipeline.computation_results[active_config_name]
active_computed_data = curr_active_pipeline.computation_results[active_config_name].computed_data
active_computation_config = curr_active_pipeline.computation_results[active_config_name].computation_config
active_computation_errors = curr_active_pipeline.computation_results[active_config_name].accumulated_errors
print(f'active_computed_data.keys(): {list(active_computed_data.keys())}')
print(f'active_computation_errors: {active_computation_errors}')
active_pf_1D = curr_active_pipeline.computation_results[active_config_name].computed_data['pf1D']
active_pf_2D = curr_active_pipeline.computation_results[active_config_name].computed_data['pf2D']    
active_pf_1D_dt = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf1D_dt', None)
active_pf_2D_dt = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_dt', None)
active_one_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_Decoder', None)
active_two_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_TwoStepDecoder', None)
active_eloy_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('EloyAnalysis', None)
active_simpler_pf_densities_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('SimplerNeuronMeetingThresholdFiringAnalysis', None)
active_ratemap_peaks_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', None)
active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
active_measured_positions = curr_active_pipeline.computation_results[active_config_name].sess.position.to_dataframe()
curr_spikes_df = sess.spikes_df

curr_active_config = curr_active_pipeline.active_configs[active_config_name]
curr_active_display_config = curr_active_config.plotting_config

display_output = dict()

active_computed_data.keys(): ['pf1D', 'pf2D', 'pf1D_dt', 'pf2D_dt', 'pf2D_Decoder', 'pf2D_TwoStepDecoder', 'EloyAnalysis', 'SimplerNeuronMeetingThresholdFiringAnalysis', 'RatemapPeaksAnalysis', 'extended_stats', 'firing_rate_trends', 'placefield_overlap']
active_computation_errors: {}


In [ ]:
list(active_computation_errors.values())[0]

In [ ]:
err_fn = list(active_computation_errors.keys())[0]
err_fn

In [ ]:
err_fn.__name__ # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'

## 3D Interactive Spike Raster Window

In [ ]:
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

display_output = display_output | curr_active_pipeline.display('_display_spike_rasters_window', active_config_name, active_config_name=active_config_name)
spike_raster_window = display_output['spike_raster_window']
# spike_raster_plt_2d = display_output['spike_raster_plt_2d']
# spike_raster_plt_3d = display_output['spike_raster_plt_3d']

In [ ]:
spike_raster_window = Spike3DRasterWindowWidget(curr_spikes_df, application_name=f'Spike Raster Window - {active_config_name}')
# spike_raster_window = Spike3DRasterWindowWidget(curr_spikes_df, neuron_colors=curr_active_display_config.pf_colors.copy(), neuron_sort_order=None, application_name=f'Spike Raster Window - {active_config_name}')
# spike_raster_window = Spike3DRasterWindowWidget(curr_spikes_df, neuron_colors=provided_neuron_id_to_color_map, neuron_sort_order=None, application_name=f'Spike Raster Window - {active_config_name}')
# spike_raster_window = Spike3DRasterWindowWidget(curr_spikes_df, neuron_colors=ipcDataExplorer.active_neuron_render_configs_map) # Test passing in an existing render configs map
# Set Window Title Options:
spike_raster_window.setWindowFilePath(str(sess.filePrefix.resolve()))
spike_raster_window.setWindowTitle(f'Spike Raster Window - {active_config_name} - {str(sess.filePrefix.resolve())}')

In [ ]:
spike_raster_window.spike_raster_plt_2d.cell_id_to_fragile_linear_neuron_IDX_map

In [ ]:
from pyphoplacecellanalysis.Pho3D.Mixins.Test import Test

In [ ]:
## Set the colors of the raster window from the tuning curve window:
spike_raster_window.update_neurons_color_data(updated_neuron_render_configs=provided_neuron_id_to_color_map)

In [ ]:
# Setup Connections Menu:
root_window, menuConnections, actions_dict = ConnectionControlsMenuMixin.try_add_connections_menu(spike_raster_window) # none of these properties need to be 

### Test building a second spike_raster_window for a different epoch:

In [ ]:
secondary_active_config_name = 'maze2'
secondary_spikes_df = curr_active_pipeline.filtered_sessions[secondary_active_config_name].spikes_df
spike_raster_window_second_epoch = Spike3DRasterWindowWidget(secondary_spikes_df, application_name=f'Spike Raster Window - {secondary_active_config_name}', type_of_3d_plotter='vedo')
# Set Window Title Options:
spike_raster_window_second_epoch.setWindowFilePath(str(sess.filePrefix.resolve()))
spike_raster_window_second_epoch.setWindowTitle(f'Spike Raster Window - {secondary_active_config_name} - {str(sess.filePrefix.resolve())}')
spike_raster_window_second_epoch

In [ ]:
spike_raster_window.spikes_df # has scISI column!


In [ ]:
spike_raster_window.connection_man

In [ ]:
spike_raster_window.connection_man.active_connections

In [ ]:
spike_raster_window.connection_man.get_available_drivers()

## ipcDataExplorer - 3D Interactive Tuning Curves Plotter

In [ ]:
pActiveTuningCurvesPlotter = None
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', active_config_name, extant_plotter=display_output.get('pActiveTuningCurvesPlotter', None), panel_controls_mode='Qt', should_nan_non_visited_elements=False, zScalingFactor=2000.0, separate_window=False) # Works now!
ipcDataExplorer = display_output['ipcDataExplorer']
display_output['pActiveTuningCurvesPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveTuningCurvesPlotter = display_output['pActiveTuningCurvesPlotter']
root_dockAreaWindow, placefieldControlsContainerWidget, pf_widgets = display_output['pane'] # for Qt mode:

In [ ]:
# ipcDataExplorer.ui.btnToggleOccupancy
# ipcDataExplorer.ui.btnNonPlacefieldSpikes
# ipcDataExplorer.spikes_df.columns # [..., 'is_pf_included', 'render_opacity', 'render_exclusion_mask', 'old_fragile_linear_neuron_IDX', 'neuron_IDX', 'rgb_hex', 'R', 'G', 'B', 'z_fixed']

# ipcDataExplorer.spikes_df['z_fixed'] = 0.0
ipcDataExplorer.change_spike_rows_included(row_specifier_mask=np.logical_not(ipcDataExplorer.spikes_df['is_pf_included'].to_numpy()), are_included=False) # Exclude the non-pf spikes

In [ ]:
ipcDataExplorer.spikes_df

In [ ]:
ipcDataExplorer.change_spike_rows_included(row_specifier_mask=np.logical_not(ipcDataExplorer.spikes_df['is_pf_included'].to_numpy()), are_included=True) # Include the non-pf spikes

In [ ]:
ipcDataExplorer.update_spikes()

In [ ]:
# ipcDataExplorer.spikes_df['render_opacity'] = ipcDataExplorer.spikes_df['render_opacity'](ipcDataExplorer.spikes_df['render_opacity'] == 1.0) np.min(ipcDataExplorer.spikes_df['render_opacity'], 0.5)
ipcDataExplorer.spike_exclusion_mask

In [ ]:
ipcDataExplorer.plots['spikes_pf_active']

In [ ]:
placefieldControlsContainerWidget

### Setup Extra Buttons

In [ ]:
ipcDataExplorer.occupancy_plotting_config.barOpacity = 0.25

In [ ]:
placefieldControlsContainerWidget.ui.end_button_helper_connections

## Optional Duplicate ipcDataExplorer plotter for comparison

In [ ]:
pActiveTuningCurvesPlotter_dup = None
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', active_config_name, extant_plotter=display_output.get('pActiveTuningCurvesPlotter_dup', None), panel_controls_mode='Qt', should_nan_non_visited_elements=False, zScalingFactor=2000.0) # Works now!
ipcDataExplorer_dup = display_output['ipcDataExplorer']
display_output['pActiveTuningCurvesPlotter_dup'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveTuningCurvesPlotter_dup = display_output['pActiveTuningCurvesPlotter_dup']
root_dockAreaWindow_dup, placefieldControlsContainerWidget_dup, pf_widgets_dup = display_output['pane'] # for Qt mode:

## Filter to include only spikes that were used in the computation of placefields:

In [ ]:
## Call update_spikes() to render the changes:
ipcDataExplorer.update_spikes()

### Render computed contours and peaks to ipcDataExplorer:

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.peak_prominences import render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter
## Call the function to add the 3D plot components to the pyvista plotter
render_all_neuron_peak_prominence_2d_results_on_pyvista_plotter(ipcDataExplorer, active_peak_prominence_2d_results, debug_print=False)

In [ ]:
## Set the colors of the raster window from the tuning curve window:
spike_raster_window.update_neurons_color_data(updated_neuron_render_configs=ipcDataExplorer.active_neuron_render_configs_map)

In [ ]:
ipcDataExplorer.p.enable_depth_peeling(10, occlusion_ratio=0) # This fixes many of the rendering issues of the placefields, but does slow things way down. 
# ipcDataExplorer.p.enable_depth_peeling(number_of_peels=4, occlusion_ratio=0)
# ipcDataExplorer.p.disable_depth_peeling()
# ipcDataExplorer.p.enable_depth_peeling(10)

In [ ]:
## Change background color:
ipcDataExplorer.params.plotter_backgrounds

In [ ]:
ipcDataExplorer.set_background('Deep Space (Dark)')

In [ ]:
ipcDataExplorer.set_background('Purple Paradise')

In [ ]:
ipcDataExplorer.plot_placefields()

In [ ]:
ipcDataExplorer.params.zScalingFactor

In [ ]:
# ipcDataExplorer.toggle_plot_visibility() # Looks like it works for generally specified plot key
# ipcDataExplorer.get_plot_objects_list

# ['main', 'points', 'peaks']
list(ipcDataExplorer.tuning_curve_plot_actors[2].keys())
# 'tuningCurvePlotActors'

In [ ]:
curr_tuning_curve_data = ipcDataExplorer.plots_data['tuningCurvePlotData'][2]
curr_pdata = curr_tuning_curve_data['pdata_currActiveNeuronTuningCurve'] # StructuredGrid
curr_pdata_points = curr_tuning_curve_data['pdata_currActiveNeuronTuningCurve_Points'] # UnstructuredGrid
curr_pdata_points
# curr_pdata
# {'curr_active_neuron_ID': 2,
#  'curr_active_neuron_pf_identifier': 'pf[2]',
#  'curr_active_neuron_tuning_Curve': array([[0.437126, 0.818305, 1.48319, ..., 0.0155238, 0.0067005, 0.00294359],
#         [0.347969, 0.64914, 1.17457, ..., 0.0388841, 0.0167967, 0.00739304],
#         [0.228976, 0.415578, 0.741638, ..., 0.0922773, 0.0399187, 0.0176301],
#         ...,
#         [0, 0, 0, ..., 0.0989772, 0.0683015, 0.0467108],
#         [0, 0, 0, ..., 0.0407058, 0.0284058, 0.0195678],
#         [0, 0, 0, ..., 0.0158926, 0.0112009, 0.00776478]]),
#  'pdata_currActiveNeuronTuningCurve': StructuredGrid (0x24d816a7f40)
#    N Cells:	1764
#    N Points:	1856
#    X Bounds:	2.581e+01, 2.638e+02
#    Y Bounds:	1.244e+02, 1.536e+02
#    Z Bounds:	0.000e+00, 1.703e+01
#    Dimensions:	29, 64, 1
#    N Arrays:	3,
#  'pdata_currActiveNeuronTuningCurve_Points': UnstructuredGrid (0x24d815b4dc0)
#    N Cells:	1716
#    N Points:	1812
#    X Bounds:	2.581e+01, 2.638e+02
#    Y Bounds:	1.244e+02, 1.536e+02
#    Z Bounds:	0.000e+00, 1.703e+01
#    N Arrays:	3,
#  'lut': <vtkmodules.vtkCommonCore.vtkLookupTable(0x0000024D9A888950) at 0x0000024DAA8F35E0>,
#  'peaks':
#  ...

surf = curr_pdata.extract_surface() # <class 'pyvista.core.pointset.PolyData'>
surf

# surf = curr_pdata_points.reconstruct_surface() # this does not work at all
# surf
# curr_pdata_points.length

# active_points_data = curr_pdata_points.points.copy()
# active_points_data = curr_pdata.copy()
active_points_data = surf.copy()
origin = active_points_data.center.copy()
origin[-1] = origin[-1] - (active_points_data.length/3.0)
origin
projected = active_points_data.project_points_to_plane(origin=origin)
projected

In [ ]:
edges = curr_pdata.extract_all_edges()
edges.plot(line_width=1, color='k')

In [ ]:
# non_visited_mask = active_placefields.never_visited_occupancy_mask
non_visited_mask = ipcDataExplorer.params.active_epoch_placefields.never_visited_occupancy_mask.copy()
non_visited_mask

In [ ]:
flat_non_visited_mask = non_visited_mask.T.copy().ravel(order="F")
# flat_non_visited_mask.shape # (1856,)

In [ ]:
# Only if VTK > 9.1.X does this return a PointData, otherwise returns None
# _ = curr_pdata.hide_points(ind=flat_non_visited_mask)
curr_pdata.hide_cells(flat_non_visited_mask, inplace=True)
# curr_updated_pdata_points = curr_pdata_points.hide_points(non_visited_mask).copy()
# curr_updated_pdata_points
# List or array of point indices to be hidden. The array can also be a boolean array of the same size as the number of points.


In [ ]:
import pyvista as pv
pl = pv.Plotter(shape=(1,2))
pl.add_mesh(curr_pdata_points)
pl.add_title('points')
pl.subplot(0,1)
pl.add_mesh(curr_pdata, color=True, show_edges=True)
pl.add_mesh(surf, color=True, show_edges=True)
pl.add_mesh(projected)
pl.add_mesh(curr_pdata)
pl.add_title('reconstructed from points')
pl.show()

In [ ]:
curr_tuning_curve_plot = ipcDataExplorer.tuning_curve_plot_actors[2]['main']
curr_tuning_curve_plot.GetShaderProperty()

In [ ]:
sess_spikes_df

## Test Programmatically Adding GUI/UI controls for additional plots (Occupancy, etc)

In [ ]:
# ipcDataExplorer.ui.placefieldControlsContainerWidget # PlacefieldVisualSelectionControlsBarWidget
# ipcDataExplorer.ui.placefieldControlsContainerWidget.ui.pf_layout # QHBoxLayout 
curr_neuron_widget = ipcDataExplorer.ui.placefieldControlsContainerWidget.ui.pf_widgets[0] # Grab single neuron widget (PlacefieldVisualSelectionWidget)
curr_layout = curr_neuron_widget.ui.groupBox.layout() # QVBoxLayout 

In [ ]:
chkbtnProminenceMetrics = curr_neuron_widget.build_ui_toggle_button(name='chkbtnProminenceMetrics', text='prominences', parent=curr_neuron_widget.ui.groupBox)
curr_layout.addWidget(chkbtnProminenceMetrics)

<!-- ### Occupancy Toggle -->

In [ ]:
# ipcDataExplorer.on_occupancy_plot_update_visibility
ipcDataExplorer.occupancy_plotting_config.isVisible = False

In [ ]:
ipcDataExplorer.plot_occupancy_bars()

In [ ]:
ipcDataExplorer.occupancy_plotting_config.labelsAreVisible = True # Setting this to true renders the labels at each occupancy bin

In [ ]:
ipcDataExplorer.occupancy_plotting_config.labelsShowPoints = True

In [ ]:
ipcDataExplorer.params.should_nan_non_visited_elements = False

In [ ]:
filtered_indices = sess_spikes_df.index.values[active_pf_2D.filtered_spikes_df.index]
# filtered_sess_spikes_df = sess_spikes_df[sess_spikes_df.index == active_pf_2D.filtered_spikes_df.index]
# filtered_sess_spikes_df = sess_spikes_df[sess_spikes_df['flat_spike_idx'] == active_pf_2D.filtered_spikes_df.flat_spike_idx]
# filtered_sess_spikes_df

In [ ]:
ipcDataExplorer.occupancy_plotting_config.isVisible = False

In [ ]:
ipcDataExplorer.occupancy_plotting_config

## ipspikesDataExplorer - 3D Interactive Spike and Behavior Plotter

In [ ]:
pActiveSpikesBehaviorPlotter = None
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', active_config_name, extant_plotter=display_output.get('pActiveSpikesBehaviorPlotter', None)) # Works now!
ipspikesDataExplorer = display_output['ipspikesDataExplorer']
display_output['pActiveSpikesBehaviorPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveSpikesBehaviorPlotter = display_output['pActiveSpikesBehaviorPlotter']

In [ ]:
## Sync ipspikesDataExplorer to raster window:
extra_interactive_spike_behavior_browser_sync_connection = spike_raster_window.connect_additional_controlled_plotter(controlled_plt=ipspikesDataExplorer)
# extra_interactive_spike_behavior_browser_sync_connection = _connect_additional_controlled_plotter(spike_raster_window.spike_raster_plt_2d, ipspikesDataExplorer)

### Test Programmatic Updating of the ipspikesDataExplorer

In [ ]:
ipspikesDataExplorer.spikes_main_historical.VisibilityOff()
# ipspikesDataExplorer.

In [ ]:
np.shape(ipspikesDataExplorer.params.flattened_spike_positions_list) # (2, 102139)
np.shape(ipspikesDataExplorer.flattened_spike_times) # (102139,)

recent_spikes_mesh = ipspikesDataExplorer.plots['spikes_main_recent_only']
# ipspikesDataExplorer.plots.keys() # dict_keys(['spikes_main_historical', 'spikes_main_recent_only', 'animal_location_trail', 'animal_current_location_point', 'maze_bg'])
# , show_edges=True

In [ ]:
ipspikesDataExplorer.spikes_main_historical.AddPosition((0.0, 0.0, (-1.1*10)))

## Extras

In [ ]:
# spike_raster_window.render_window_duration
# spike_raster_window.spikes_window.active_window_end_time
spike_raster_window.spikes_window.window_duration

In [ ]:
### Debug printing for color properties of spike_raster_window
_debug_params = spike_raster_window.spike_raster_plt_2d.params
# _debug_params.neuron_colors
# _debug_params.config_items
# spike_raster_window.params

In [ ]:
# Wanted to try to set setClickable(True) on the children widgets

# spike_raster_window.spike_raster_plt_2d.plots # {'name': '', 'preview_overview_scatter_plot': <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.ScatterPlotItem.ScatterPlotItem object at 0x0000020AA96D9E50>}
# spike_raster_window.spike_raster_plt_2d.plots.main_plot_widget # <pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.PlotItem.PlotItem.PlotItem at 0x20aecdf0ee0>
# spike_raster_window.spike_raster_plt_2d.plots.main_plot_widget.curves # [<pyphoplacecellanalysis.External.pyqtgraph.graphicsItems.ScatterPlotItem.ScatterPlotItem at 0x20aa92a5820>]

# Static Background Overview Scatter Plot:
active_overview_scatter_plot = spike_raster_window.spike_raster_plt_2d.plots.preview_overview_scatter_plot # ScatterPlotItem 
# active_overview_scatter_plot.addPoints(hoverable=True)

## Main Scatter Plot 
main_scatter_plot = spike_raster_window.spike_raster_plt_2d.plots.main_plot_widget.curves[0] # ScatterPlotItem 
# main_scatter_plot.setAcceptHoverEvents(True)

# Common Tick Label
vtick = QtGui.QPainterPath()
vtick.moveTo(0, -0.5)
vtick.lineTo(0, 0.5)

# # Highlights the hovered spikes white:
# main_scatter_plot.addPoints(hoverable=True,
#     hoverSymbol=vtick, # hoverSymbol='s',
#     hoverSize=7, # default is 5
#     hoverPen=pg.mkPen('w', width=2),
#     hoverBrush=pg.mkBrush('w'))

# Highlights the hovered spikes white:
main_scatter_plot.addPoints(hoverable=True,
    # hoverSymbol=vtick, # hoverSymbol='s',
    hoverSize=7, # default is 5
    )


## Clickable/Selectable Spikes:
# Will make all plots clickable
clickedPen = pg.mkPen('#DDD', width=2)
lastClicked = []
def _test_scatter_plot_clicked(plot, points):
    global lastClicked
    for p in lastClicked:
        p.resetPen()
    print("clicked points", points)
    for p in points:
        p.setPen(clickedPen)
    lastClicked = points
        
## Hoverable Spikes:
def _test_scatter_plot_hovered(plt, points, ev):
    # sigHovered(self, points, ev)
    print(f'_test_scatter_plot_hovered(plt: {plt}, points: {points}, ev: {ev})')
    if (len(points) > 0):
        curr_point = points[0]
        # self.
        # curr_point.index

main_scatter_hovered_connection = main_scatter_plot.sigHovered.connect(_test_scatter_plot_hovered)
main_scatter_clicked_connection = main_scatter_plot.sigClicked.connect(_test_scatter_plot_clicked)

# tip
# .sigHovered 
# TypeError: 'VisualizationParameters' object is not subscriptable

In [ ]:
## Test Adding ROIs of interest to the 2D Raster Plot:
rois = []
rois.append(pg.MultiRectROI([[20, 90], [50, 60], [60, 90]], width=5, pen=(2,9)))

def update(roi):
    img1b.setImage(roi.getArrayRegion(arr, img1a), levels=(0, arr.max()))
    v1b.autoRange()
    
for roi in rois:
    roi.sigRegionChanged.connect(update)
    v1a.addItem(roi)

In [ ]:
curve.curve.setClickable(True)

## Updated Color Config Handling

In [ ]:
ipcDataExplorer.active_neuron_render_configs

In [ ]:
# [SingleNeuronPlottingExtended(color='#843c39', extended_values_dictionary={}, isVisible=False, name='2', spikesVisible=False),
#  SingleNeuronPlottingExtended(color='#9d514e', extended_values_dictionary={}, isVisible=False, name='3', spikesVisible=False),
#  ...
# ]

In [ ]:
test_updated_configs_colors_map[3] = '#333333'

In [ ]:
# Test applying changes to configs:
active_configs_map, updated_ids_list, updated_configs_list = _test_apply_updated_colors_map_to_configs(active_configs_map, test_updated_configs_colors_map)
# active_configs_map

In [ ]:
## Test applying it to the Qt widgets:
# ipcDataExplorer.find_rows_matching_cell_ids

ipcDataExplorer.active_neuron_render_configs

In [ ]:
test_updated_configs_colors_map

In [ ]:
# updated_ids_list
updated_configs_list

In [ ]:
# ipcDataExplorer.update_spikes()
# ipcDataExplorer.gui.
# placefieldControlsContainerWidget

ipcDataExplorer.neuron_id_pf_widgets_map = _build_id_index_configs_dict(pf_widgets)
apply_updated_configs_to_pf_widgets(ipcDataExplorer.neuron_id_pf_widgets_map, active_configs_map)


# neuron_id_pf_widgets_map

# for a_pf_ctrl_widget in pf_widgets:
#     print(f'{a_pf_ctrl_widget.name}')
#     # int(a_config.name)

In [ ]:
test_updated_colors_map = {3: '#999999'}
ipcDataExplorer.on_config_update(test_updated_colors_map)

In [ ]:
# Need to rebuild the spikes colors and such upon updating the configs. 
# should take a config and produce the changes needed to recolor the neurons.
ipcDataExplorer.update_spikes_df_color_columns(test_updated_colors_map)
ipcDataExplorer.update_rendered_placefields(test_updated_colors_map)

In [ ]:
_test_neuron_id = 3
pdata_currActiveNeuronTuningCurve_plotActor = ipcDataExplorer.plots['tuningCurvePlotActors'][_test_neuron_id]['main'] # get the main plot actor from the CascadingDynamicPlotsList
pdata_currActiveNeuronTuningCurve_Points_plotActor = ipcDataExplorer.plots['tuningCurvePlotActors'][_test_neuron_id]['points']

# add_mesh

In [ ]:
ipcDataExplorer.enable_overwrite_invalid_fragile_linear_neuron_IDXs = True

In [ ]:
ipcDataExplorer.helper_setup_neuron_colors_and_order(ipcDataExplorer, debug_print=True)

In [ ]:
from vedo.io import exportWindow, write
# exportWindow('testOutFilePho.html')
# exportWindow('testOutFilePho.x3d')

# pActiveTuningCurvesPlotter.save('testPlotterSavePho.obj')
# write(pActiveTuningCurvesPlotter,'testPlotterSavePho.obj')
# pActiveTuningCurvesPlotter.write_frame('testPlotter_write_frame_Pho')
# pActiveTuningCurvesPlotter.export('testPlotterSavePho.x3d')

output_path = Path(r'C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\output\2006-6-07_11-26-53\maze1')
final_output_path = output_path.joinpath('testPlotter_export_obj_Pho')


## Success, but ugly:
pActiveTuningCurvesPlotter.export_vtkjs(final_output_path.with_suffix('.vtkjs'))
# pActiveTuningCurvesPlotter.export_gltf(final_output_path.with_suffix('.gltf'))
# pActiveTuningCurvesPlotter.export_html(final_output_path.with_suffix('.html')) ## First export that actually works with BackgroundPlotter
# pActiveTuningCurvesPlotter.export_obj(final_output_path.with_suffix('.obj')) ## Works for exporting .obj files

In [ ]:
import pyvista as pv
pv.vtk_version_info

In [ ]:
# ipcDataExplorer.p.enable_cell_picking(callback=)
ipcDataExplorer.pyqtConfigure()

In [ ]:
p.enable_point_picking()

In [ ]:
ipcDataExplorer.active_config # pf_neuron_identities, pf_colors, pf_sort_ind

# ipcDataExplorer.active_config.keys()
np.shape(ipcDataExplorer.active_config.plotting_config['pf_colormap']) # (39, 4)

# print_keys_if_possible('plotting_config', ipcDataExplorer.active_config.plotting_config)

print_keys_if_possible('ipcDataExplorer', ipcDataExplorer)

# - plotting_config: <class 'pyphoplacecellanalysis.General.Configs.DynamicConfigs.PlottingConfig'>
    # - pf_neuron_identities: <class 'list'> - (39,)
    # - pf_sort_ind: <class 'numpy.ndarray'> - (39,)
    # - pf_colors: <class 'numpy.ndarray'> - (4, 39)
    # - pf_colormap: <class 'numpy.ndarray'> - (39, 4)

In [ ]:
len(ipcDataExplorer.params.neuron_colors_hex) # 39

In [ ]:
# np.shape(ipcDataExplorer.params.neuron_colors) # (4, 39)
# len(ipcDataExplorer.params.reverse_cellID_to_tuning_curve_idx_lookup_map) # 39

# for neuron_id, tuning_curve_idx in ipcDataExplorer.params.reverse_cellID_to_tuning_curve_idx_lookup_map.items():
#     curr_color = ipcDataExplorer.params.neuron_colors[:, tuning_curve_idx]
    


final_perfect_colormap = {int(neuron_id):ipcDataExplorer.params.neuron_colors_hex[tuning_curve_idx] for neuron_id, tuning_curve_idx in ipcDataExplorer.params.reverse_cellID_to_tuning_curve_idx_lookup_map.items()}
# final_perfect_colormap = {int(neuron_id):list(np.squeeze(ipcDataExplorer.params.neuron_colors[:, tuning_curve_idx])) for neuron_id, tuning_curve_idx in ipcDataExplorer.params.reverse_cellID_to_tuning_curve_idx_lookup_map.items()}
# final_perfect_colormap = {int(neuron_id):QtGui.QColor(*np.squeeze(ipcDataExplorer.params.neuron_colors[:, tuning_curve_idx])) for neuron_id, tuning_curve_idx in ipcDataExplorer.params.reverse_cellID_to_tuning_curve_idx_lookup_map.items()}
final_perfect_colormap

In [ ]:
len(final_perfect_colormap) # 39

In [ ]:
ipcDataExplorer.on_config_update(updated_colors_map=final_perfect_colormap)

In [ ]:
partial_perfect_colormap = {24: [0.9098039215686274, 0.8075259642318465, 0.6095686274509803, 1.0],
 25: [1.0, 0.9148351648351648, 0.75, 1.0],
 26: [0.388235294117647, 0.4745098039215686, 0.22352941176470587, 1.0],
 27: [0.48533923796791445, 0.5796078431372549, 0.3053719008264463, 1.0]}
ipcDataExplorer.on_update_spikes_colors(neuron_id_color_update_dict=partial_perfect_colormap)

In [ ]:
QtGui.QColor(*(0.5176470588235295, 0.23529411764705882, 0.22352941176470587, 1.0))

In [ ]:
# don't use this one!
len(ipcDataExplorer.params.reverse_cellID_idx_lookup_map) # 40

In [ ]:
ipcDataExplorer.active_neuron_render_configs # confirmed same configs as active_tuning_curve_render_configs

In [ ]:
# ipcDataExplorer.setup_spike_rendering_mixin()

ipcDataExplorer.spikes_df # first many entries all look like R G B columns are 0.0 even if rgb_hex isn't and is different!

# np.count_nonzero(ipcDataExplorer.spikes_df.R.to_numpy()) # 5989
ipcDataExplorer.spikes_df.columns

ipcDataExplorer.spikes_df[['aclu', 'fragile_linear_neuron_IDX', 'neuron_IDX']] ## BREAKTHROUGH: 'fragile_linear_neuron_IDX' and 'neuron_IDX' are definitely not equal (but I think they should be)

In [ ]:
np.unique(ipcDataExplorer.spikes_df['fragile_linear_neuron_IDX'])
## MAJOR CONCERN: 'fragile_linear_neuron_IDX' values make no sense at all. They aren't even equal to 'aclu'
# array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 12, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 31, 32, 36, 37, 40, 42, 43, 44, 45, 46, 51, 53, 55, 56, 59, 60, 61, 62])

In [ ]:
np.unique(ipcDataExplorer.spikes_df['neuron_IDX'])

In [ ]:
ipcDataExplorer.spikes_df['old_fragile_linear_neuron_IDX']

In [ ]:
ipcDataExplorer.spikes_df[['aclu', 'fragile_linear_neuron_IDX', 'neuron_IDX', 'old_fragile_linear_neuron_IDX']] ## GOOD: after rebuilding the 'fragile_linear_neuron_IDX' and 'neuron_IDX' columns now match as expected.

In [ ]:
ipcDataExplorer.neuron_ids

In [ ]:
# ipcDataExplorer.update_active_spikes(
ipcDataExplorer.update_spikes()

In [ ]:
## Rebuild the IDXs
ipcDataExplorer.spikes_df.spikes._obj, neuron_id_to_new_IDX_map_new_method = ipcDataExplorer.spikes_df.spikes.rebuild_fragile_linear_neuron_IDXs(debug_print=True)
new_neuron_IDXs = list(neuron_id_to_new_IDX_map_new_method.values())

In [ ]:
ipcDataExplorer.params.cell_spike_opaque_colors_dict # this is what spikes_df's 'R', 'G', 'B' columns set from.
# keys are neuron_IDX format

In [ ]:
# Disable the bar on the docks:
dDisplayItem1 = root_dockAreaWindow.dynamic_display_dict['Dock1 - Controls']['Dock1 - Controls']['dock']
dDisplayItem2 = root_dockAreaWindow.dynamic_display_dict['Dock2 - Content']['Dock2 - Content']['dock']

dDisplayItem1.hideTitleBar()
dDisplayItem2.hideTitleBar()

In [ ]:
ipcDataExplorer.params.end_button_helper_obj.btn_show_all_callback(True)

In [ ]:
# ipcDataExplorer.update_tuning_curve_configs()
ipcDataExplorer.update_neuron_render_configs()

In [ ]:
# ipcDataExplorer.params.cell_spike_colors_dict
# ipcDataExplorer.active_neuron_render_configs

active_placefields = ipcDataExplorer.params.active_epoch_placefields
# ipcDataExplorer.params.pf_color
pf_colors = ipcDataExplorer.params.pf_colors
if np.shape(pf_colors)[1] > 3:
    opaque_pf_colors = pf_colors[0:3,:].copy() # get only the RGB values, discarding any potnential alpha information
else:
    opaque_pf_colors = pf_colors.copy()

occupancy = active_placefields.ratemap.occupancy.copy()
# curr_tuning_curves = active_placefields.ratemap.tuning_curves.copy() # (39, 59, 21)
curr_tuning_curves = active_placefields.ratemap.normalized_tuning_curves.copy() # (39, 59, 21)
good_placefield_neuronIDs = ipcDataExplorer.params.pf_unit_ids
# np.shape(curr_tuning_curves) # (39, 59, 21)
# np.isnan(curr_tuning_curves)
# np.count_nonzero(np.isnan(curr_tuning_curves))


# Never Visited Mask
# occupancy

In [ ]:
# never_visited_occupancy_mask
i = 0
curr_active_neuron_ID = good_placefield_neuronIDs[i]
curr_active_neuron_color = pf_colors[:, i]
curr_active_neuron_opaque_color = opaque_pf_colors[:,i]
curr_active_neuron_pf_identifier = 'pf[{}]'.format(curr_active_neuron_ID)
curr_active_neuron_tuning_Curve = np.squeeze(curr_tuning_curves[i,:,:]).T.copy() # A single tuning curve, (21, 59)
# curr_active_neuron_tuning_Curve

In [ ]:
curr_active_plot_actor = ipcDataExplorer.plots['tuningCurvePlotActors'][i]
curr_active_plot_data = ipcDataExplorer.plots_data['tuningCurvePlotData'][i]
curr_active_plot_data
pdata_currActiveNeuronTuningCurve = curr_active_plot_data['pdata_currActiveNeuronTuningCurve'] # StructuredGrid

# Test Extracting Points:
curr_mesh_extracted_pts = pdata_currActiveNeuronTuningCurve.extract_points(pdata_currActiveNeuronTuningCurve.points[:, 2] > 0)
type(curr_mesh_extracted_pts) # UnstructuredGrid

In [ ]:
# Add labels to points on the yz plane (where x == 0)
points = pdata_currActiveNeuronTuningCurve.points
# mask = points[:, 0] == 0
mask = points[:, 2] > 0.1
pActiveTuningCurvesPlotter.add_point_labels(points[mask], points[mask].tolist(), point_size=20, font_size=36)

In [ ]:
np.shape(never_visited_occupancy_mask) # (59, 21)
np.sum(curr_active_neuron_tuning_Curve[~never_visited_occupancy_mask.T]) # (735,), 7.9040396441924194
curr_active_neuron_tuning_Curve[~never_visited_occupancy_mask.T] = np.nan # set non-visited locations to NaN
# NOTE: the sum of all visited locations is 36.44356525201446 and those non-visited locations is 7.9040396441924194
curr_active_neuron_tuning_Curve

In [ ]:
curr_active_pipeline.active_configs[active_config_name].keys()

In [ ]:

updated_video_output_config = curr_active_pipeline.active_configs[active_config_name]['video_output_config']
updated_video_output_config.active_frame_range = np.arange(100, 480, 1)
updated_video_output_config

In [ ]:
# Enable video output by setting: active_is_video_output_mode
updated_video_output_config.active_is_video_output_mode = True


In [ ]:
import pyvista as pv
from pyvista import examples
# Download skybox
cubemap = examples.download_sky_box_cube_map()
ipspikesDataExplorer.p.add_actor(cubemap.to_skybox())
ipspikesDataExplorer.p.set_environment_texture(cubemap)  # For reflecting the environment off the mesh

In [ ]:
ipspikesDataExplorer.p.enable_shadows()

In [ ]:
ipspikesDataExplorer.p.render()

In [ ]:
# recent_spikes_mesh.
ipspikesDataExplorer.params.longer_spikes_window.duration_seconds = 10.0

In [ ]:
ipspikesDataExplorer.p.enable_depth_peeling(number_of_peels=6, occlusion_ratio=0) # Supposedly helps with translucency

In [ ]:
# spike_raster_window.spikes_window.on_window_changed()
# spike_raster_window.spikes_window.update_window_start(500.0) # doesn't update anything
# spike_raster_window.spike_raster_plt_2d.spikes_window.update_window_start(500.0) # doesn't update anything
spike_raster_window.spike_raster_plt_2d.update_scroll_window_region(500.0, 700.0, block_signals=False)

In [ ]:
spike_raster_window.shift_animation_frame_val(shift_frames=5)

In [ ]:
# spike_raster_window.spike_3d_to_2d_window_connection.disconnect(spike_raster_window.spike_raster_plt_3d.spikes_window.update_window_start_end)
spike_raster_window.spike_raster_plt_2d.window_scrolled.disconnect(spike_raster_window.spike_raster_plt_3d.spikes_window.update_window_start_end)

In [ ]:
spike_raster_window.spike_raster_plt_2d.window_scrolled.disconnect()

In [ ]:
spike_raster_window.spike_raster_plt_2d.ui.scroll_window_region.sigRegionChanged.disconnect()

In [ ]:
## Need to the spike indicies that are currently visible in the raster window to programmatically update ipspikesDataExplorer's displayed spikes.
# active_epoch_session.flattened_spiketrains.time_slice(curr_lap_spike_t_seconds.values[0], curr_lap_spike_t_seconds.values[-1]).spikes_df
# spike_raster_window.spike_raster_plt_2d.spikes_window
# curr_win_start, curr_win_end = spike_raster_window.spike_raster_plt_3d.spikes_window.active_time_window
# spike_raster_window.spike_raster_plt_3d.spikes_window.active_windowed_df

# spike_raster_window.spike_raster_plt_3d.spikes_window.active_windowed_df.index
# ipspikesDataExplorer.params.curr_view_window_length_samples # 299
# np.shape(ipspikesDataExplorer.params.pre_computed_window_sample_indicies) # (51157, 299)

# np.shape(ipspikesDataExplorer.params.active_trail_opacity_values) # (299,)
## Hopefully ipspikesDataExplorer's slider will adjust automatically?

In [ ]:
## Test disabling the user-slider:
# ipspikesDataExplorer.interface_properties
# interactive_timestamp_slider_actor
# add_slider_widget

## When building widgets:
# self.p.add_callback(self.interface_properties, interval=16)  # to be smooth on 60Hz

# Removes all slider widgets:
# ipspikesDataExplorer.p.clear_slider_widgets()
# ipspikesDataExplorer.p.clear_button_widgets() # removes the play/pause toggle checkbox so that it can be driven externally
# ipspikesDataExplorer.p.update()
# ipspikesDataExplorer.p.render()

# ipspikesDataExplorer.p.button_widgets
# ipspikesDataExplorer.p.receivers()

# For the callback, it looks like I can check the timer here:
callback_timer = ipspikesDataExplorer.p._callback_timer  # QTimer
callback_timer.isActive()
# callback_timer.isSignalConnected()
# callback_timer.stop()

ipspikesDataExplorer

In [ ]:
spike_raster_window.animation_active_time_window.window_duration

In [ ]:
# Get the times that fall within the current plot window:
curr_win_start, curr_win_end = spike_raster_window.spike_raster_plt_3d.spikes_window.active_time_window
# np.shape(ipspikesDataExplorer.t) # (51455,)

# active_window_sample_indicies
active_included_all_window_position_indicies = ((ipspikesDataExplorer.t > curr_win_start) & (ipspikesDataExplorer.t < curr_win_end)) # Two Sided Range Mode
# active_included_all_window_position_indicies
print(f'np.shape(active_included_all_window_position_indicies): {np.shape(active_included_all_window_position_indicies)}') # (51455,)

# active_included_all_window_position_indicies = np.where(active_included_all_window_position_indicies) # was a boolean area, but get the indices where true instead
active_included_all_window_position_indicies = np.squeeze(active_included_all_window_position_indicies.nonzero()) # was a boolean area, but get the indices where true instead.  (1106,)
print(f'np.shape(active_included_all_window_position_indicies): {np.shape(active_included_all_window_position_indicies)}; active_included_all_window_position_indicies: {active_included_all_window_position_indicies}')

active_num_samples = len(active_included_all_window_position_indicies)

## TODO: I think active_included_all_window_position_indicies better be the same length as .params.active_trail_opacity_values and .params.active_trail_size_values
# ipspikesDataExplorer.params.curr_view_window_length_samples

max_num_samples = ipspikesDataExplorer.params.curr_view_window_length_samples # 299
if active_num_samples > max_num_samples:
    print(f'len(active_included_all_window_position_indicies) ({active_num_samples}) > max_num_samples ({max_num_samples}). Cutting.')
    active_included_all_window_position_indicies = active_included_all_window_position_indicies[-max_num_samples:] # get only the last (meaning most recent) max_num_samples samples from the indicies that should be displayed
    active_num_samples = max_num_samples # cut down to the max number of samples
    
print(f'np.shape(active_included_all_window_position_indicies): {np.shape(active_included_all_window_position_indicies)}, active_num_samples: {active_num_samples}') # np.shape(active_included_all_window_position_indicies): (1, 1106), active_num_samples: 1    

# print(f'np.shape(active_included_all_window_position_indicies): {np.shape(active_included_all_window_position_indicies)}, active_num_samples: {active_num_samples}')    
# ipspikesDataExplorer.x[active_included_all_window_position_indicies], ipspikesDataExplorer.y[active_included_all_window_position_indicies], ipspikesDataExplorer.z_fixed[-active_num_samples:]

## Animal Position and Location Trail Plotting:
ipspikesDataExplorer.perform_plot_location_trail('animal_location_trail', ipspikesDataExplorer.x[active_included_all_window_position_indicies], ipspikesDataExplorer.y[active_included_all_window_position_indicies], ipspikesDataExplorer.z_fixed[-active_num_samples:],
                                     trail_fade_values=ipspikesDataExplorer.params.active_trail_opacity_values, trail_point_size_values=ipspikesDataExplorer.params.active_trail_size_values,
                                     render=False)

## Animal Current Position:
curr_animal_point = [ipspikesDataExplorer.x[active_included_all_window_position_indicies[-1]], ipspikesDataExplorer.y[active_included_all_window_position_indicies[-1]], ipspikesDataExplorer.z_fixed[-1]]
ipspikesDataExplorer.perform_plot_location_point('animal_current_location_point', curr_animal_point, render=False)



# curr_time_fixedSegments = ipspikesDataExplorer.t[active_window_sample_indicies] # New Way

In [ ]:
curr_win_start, curr_win_end = spike_raster_window.spike_raster_plt_3d.spikes_window.active_time_window
ipspikesDataExplorer.on_active_window_update_mesh(curr_win_start, curr_win_end, enable_position_mesh_updates=True, render=True)

# ipspikesDataExplorer.on_programmatic_data_update(self, active_included_all_historical_indicies=None, active_included_recent_only_indicies=None, active_window_sample_indicies=None, curr_animal_point=None)

### Lap Example of programmatic updating:
# curr_lap_id = 0

# curr_lap_t_start, curr_lap_t_stop = get_lap_times(curr_lap_id)
# curr_lap_subsession = lap_specific_subsessions[curr_lap_id]
# curr_lap_dataframe = lap_specific_dataframes[curr_lap_id] # the subset dataframe for this lap
# curr_lap_spike_t_seconds = curr_lap_dataframe.t_seconds
# curr_lap_spike_indicies = lap_spike_indicies[curr_lap_id] # all spike indicies that occured within the lap
# curr_lap_position_traces = curr_lap_subsession.position.traces # the animal positions that were traversed during this lap

# curr_lap_num_spikes = len(curr_lap_spike_indicies)
# print('lap[{}]: ({}, {}): {} spikes.'.format(curr_lap_id, curr_lap_t_start, curr_lap_t_stop, curr_lap_num_spikes))

# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_indicies, active_included_recent_only_indicies=curr_lap_spike_indicies) # index 145937 is out of bounds for axis 0 with size 19647
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_t_seconds, active_included_recent_only_indicies=curr_lap_spike_t_seconds) # 
# ipspikesDataExplorer.on_programmatic_data_update(active_included_all_historical_indicies=curr_lap_spike_t_seconds.values, active_included_recent_only_indicies=curr_lap_spike_t_seconds.values) # 

In [ ]:
## Can Programmaticallyt set the visibility on the different plotted elements:
ipspikesDataExplorer.toggle_plot_visibility('spikes_main_historical')
# ipspikesDataExplorer.toggle_plot_visibility('spikes_main_recent_only')
# ipspikesDataExplorer.toggle_plot_visibility('animal_location_trail')
# ipspikesDataExplorer.toggle_plot_visibility('animal_current_location_point')
# toggle_visibility(ipspikesDataExplorer.spikes_main_recent_only)
# toggle_visibility(ipspikesDataExplorer.animal_location_trail)
# toggle_visibility(ipspikesDataExplorer.animal_current_location_point)

In [ ]:
import pyvista as pv
from pyvistaqt import BackgroundPlotter

## Test Adding Points:


# curr_mesh_extracted_pts # UnstructuredGrid
curr_points_actor = curr_mesh_extracted_pts.plot(show_edges=True, line_width=3, point_size=20, )

pl = BackgroundPlotter()
curr_points_actor = pl.add_points(curr_mesh_extracted_pts, render_points_as_spheres=True, point_size=2.0, color='black')
pl.show()


## Explore Mesh/Cell Picking in ipcDataExplorer

In [ ]:
## Make the legend pickable:
import pyvista as pv

def callback(point):
    """Create a cube and a label at the click point."""
    mesh = pv.Cube(center=point, x_length=0.05, y_length=0.05, z_length=0.05)
    ipcDataExplorer.p.add_mesh(mesh, style='wireframe', color='r')
    ipcDataExplorer.p.add_point_labels(point, [f"{point[0]:.2f}, {point[1]:.2f}, {point[2]:.2f}"])


ipcDataExplorer.p.enable_surface_picking(callback=callback, left_clicking=True, show_point=False)
# ipcDataExplorer.p.enable_surface_picking(

In [ ]:
def mesh_picked_callback(mesh):
    """TODO: highlight the mesh or something - label at the click point."""
    print(f'mesh_picked_callback(mesh: {mesh})')
    # mesh = pv.Cube(center=point, x_length=0.05, y_length=0.05, z_length=0.05)
    # ipcDataExplorer.p.add_mesh(mesh, style='wireframe', color='r')
    # ipcDataExplorer.p.add_point_labels(point, [f"{point[0]:.2f}, {point[1]:.2f}, {point[2]:.2f}"])

ipcDataExplorer.p.enable_mesh_picking(callback=mesh_picked_callback, left_clicking=True, show=True)

### Other

In [ ]:
# Here we look at the population burst events for each epoch ('maze1' vs. 'maze2')
# curr_active_pipeline.sess.

# get only the spikes that occur during PBEs:
pbe_only_spikes_df = sess.spikes_df[(sess.spikes_df.PBE_id > -1)]
pbe_only_spikes_df

curr_active_pipeline.sess.pbe #[10960 rows x 4 columns]

In [ ]:
app, win, w = curr_active_pipeline.display('_display_placemaps_pyqtplot_2D', active_config_name)
win.show(); pg.exec()

# Individual Plotting Tests:

## Old Individual Plotting Functions:

In [ ]:
import matplotlib.pyplot as plt
# display matplotlib figures in a qt backed window. Works in Jupyter notebooks

In [ ]:
%matplotlib --list # Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']

In [ ]:
%matplotlib qt
## NOTE THAT ONCE THIS IS SET TO qt, it cannot be undone!

In [ ]:
enable_saving_to_disk = True
enable_saving_to_disk = False

In [ ]:
from neuropy.plotting.ratemaps import enumTuningMap2DPlotVariables # for getting the variant name from the dict

In [ ]:
filter_name = 'maze1'
# curr_bapun_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, subplots=(20, 8), max_screen_figure_size=(2256, 2048), enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, fignum=0) # works!
curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, subplots=(20, 8), max_screen_figure_size=(2256, 2048), enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=10) # works!

In [ ]:
filter_name = 'maze2'
# curr_bapun_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, subplots=(20, 8), max_screen_figure_size=(2256, 2048), enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, fignum=0) # works!
curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, subplots=(20, 8), max_screen_figure_size=(2256, 2048), enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=11) # works!

In [ ]:
filter_name = 'maze1'
# curr_kdiba_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, fignum=0, max_screen_figure_size=(None, 1868), debug_print=False, enable_saving_to_disk=enable_saving_to_disk) # works!
curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=1, max_screen_figure_size=(None, 1868), debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
filter_name = 'maze2'
# curr_kdiba_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, fignum=0, max_screen_figure_size=(None, 1868), debug_print=False, enable_saving_to_disk=enable_saving_to_disk) # works!
curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=1, max_screen_figure_size=(None, 1868), debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
filter_name = 'maze'
# curr_kdiba_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.FIRING_MAPS, fignum=0, max_screen_figure_size=(None, 1868), debug_print=False, enable_saving_to_disk=enable_saving_to_disk) # works!
curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', filter_name, enable_spike_overlay=False, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=1, max_screen_figure_size=(None, 1868), debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
# curr_kdiba_pipeline.display(DefaultDisplayFunctions._display_3d_interactive_custom_data_explorer, 'maze1') # works!
curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', 'maze1') # works!

In [ ]:
# TODO: WARNING: creates new figures, which means many open windows when using %matplotlib qt
curr_active_pipeline.display(DefaultDisplayFunctions._display_1d_placefield_validations, 'maze1') # works!

## Systematic Display Function Testing

In [ ]:
curr_active_pipeline.reload_default_display_functions()
print(curr_active_pipeline.registered_display_function_names)

In [ ]:
curr_active_pipeline.registered_display_function_names # ['_display_1d_placefield_validations', '_display_2d_placefield_result_plot_ratemaps_2D', '_display_2d_placefield_result_plot_raw', '_display_normal', '_display_placemaps_pyqtplot_2D', '_display_decoder_result', '_display_plot_most_likely_position_comparisons', '_display_two_step_decoder_prediction_error_2D', '_display_two_step_decoder_prediction_error_animated_2D', '_display_spike_rasters_pyqtplot_2D', '_display_spike_rasters_pyqtplot_3D', '_display_spike_rasters_pyqtplot_3D_with_2D_controls', '_display_spike_rasters_vedo_3D', '_display_spike_rasters_vedo_3D_with_2D_controls', '_display_spike_rasters_window', '_display_speed_vs_PFoverlapDensity_plots', '_display_3d_image_plotter', '_display_3d_interactive_custom_data_explorer', '_display_3d_interactive_spike_and_behavior_browser', '_display_3d_interactive_tuning_curves_plotter']

In [ ]:
## Test getting figure save paths:
_test_fig_path = curr_active_config.plotting_config.get_figure_save_path('test')
_test_fig_path.exists()

### Matplotlib-based plots:

In [ ]:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf, backend_pgf, backend_ps
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib.path import Path
# import matplotlib.patches as patches
# from matplotlib.patches import Rectangle
# from matplotlib.pyplot import cm
from PendingNotebookCode import _build_programmatic_display_function_testing_pdf_metadata

filter_name = active_config_name
_build_pdf_pages_output_info, programmatic_display_function_testing_output_parent_path = _build_programmatic_display_function_testing_pdf_metadata(curr_active_pipeline, filter_name=filter_name,
                                                                                                                                                   out_path=Path(r'C:\Users\pho\repos\PhoPy3DPositionAnalysis2021\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2022-07-19'))
reveal_in_system_file_manager(programmatic_display_function_testing_output_parent_path)

In [ ]:
%%capture
#The above hides the output in JupyterLab Notebooks, must be first line in the cell to hide the output of
curr_display_function_name = '_display_1d_placefield_validations'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    # plt.ioff() # disable displaying the plots inline in the Jupyter-lab notebook. NOTE: does not work in Jupyter-Lab, figures still show
    plots = curr_active_pipeline.display(curr_display_function_name, active_config_name) # works, but generates a TON of plots!
    # plt.ion()
    for fig_idx, a_fig in enumerate(plots):
        # print(f'saving fig: {fig_idx+1}/{len(plots)}')
        pdf.savefig(a_fig)
        # pdf.savefig(a_fig, transparent=True)
    # When no figure is specified the current figure is saved
    # pdf.savefig()

In [ ]:
%%capture
curr_display_function_name = '_display_2d_placefield_result_plot_raw'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    a_fig = curr_active_pipeline.display(curr_display_function_name, filter_name)
    pdf.savefig(a_fig)

# Issue with unexpected kwargs passed to plots. Solve generally.
# TypeError: plot_raw() got an unexpected keyword argument 'enable_saving_to_disk'

In [ ]:
# %%capture
curr_display_function_name = '_display_normal'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    fig0, figList1 = curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk)
    plots = [fig0, *figList1]
    for a_fig in plots:
        pdf.savefig(a_fig)

In [ ]:
%%capture
curr_display_function_name = '_display_decoder_result'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
    print(plots)
    # pdf.savefig(a_fig)

### PyQtGraph-based plots:

In [ ]:
# import pyqtgraph as pg
import pyphoplacecellanalysis.External.pyqtgraph.exporters
# import pyphoplacecellanalysis.External.pyqtgraph.widgets.GraphicsLayoutWidget
from pyphoplacecellanalysis.External.pyqtgraph.widgets.GraphicsView import GraphicsView

def export_pyqtgraph_plot(a_plot, debug_print=True):
    # create an exporter instance, as an argument give it
    # the item you wish to export    
    if isinstance(a_plot, GraphicsView):
        a_plot = a_plot.scene()
    else:
        a_plot = a_plot.plotItem
    exporter = pg.exporters.ImageExporter(a_plot)
    # set export parameters if needed
    # exporter.parameters()['width'] = 100   # (note this also affects height parameter)
    # save to file
    export_filepath = 'fileName.png'
    exporter.export(export_filepath)
    if debug_print:
        print(f'exported plot to {export_filepath}')


In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n_and_x_prev') # works!

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_animated_2D', active_config_name, variable_name='p_x_given_n')

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n') # works!

In [ ]:
curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', active_config_name) # this works now!

In [ ]:
curr_display_function_name = '_display_spike_rasters_pyqtplot_2D'
curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
# out_plot_1D, out_plot_2D = curr_active_pipeline.display('_display_speed_vs_PFoverlapDensity_plots', active_config_name)
curr_display_function_name = '_display_speed_vs_PFoverlapDensity_plots'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
plots

In [ ]:
curr_display_function_name = '_display_placemaps_pyqtplot_2D'
out_plots = curr_active_pipeline.display(curr_display_function_name, filter_name)    
out_plots[1].show()

In [ ]:
curr_display_function_name = '_display_speed_vs_PFoverlapDensity_plots'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
plots

In [ ]:
curr_display_function_name = 'display_firing_rate_trends'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)

In [ ]:
# a_plot = plots[0] # PlotWidget 
# a_plot_item = a_plot.plotItem # PlotItem
# a_plot.scene() # GraphicsScene
export_pyqtgraph_plot(plots[0])

### 3D (Vedo/etc)-based plots:

In [ ]:
display_dict = curr_active_pipeline.display('_display_3d_interactive_custom_data_explorer', active_config_name) # does not work, missing color info?
iplapsDataExplorer = display_dict['iplapsDataExplorer']
# plotter is available at
p = display_dict['plotter']
iplapsDataExplorer

In [ ]:
should_smooth_maze

# Common: Optional Saving

In [ ]:
import pickle
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import _test_save_pipeline_data_to_h5, get_h5_data_keys, save_some_pipeline_data_to_h5, load_pipeline_data_from_h5  #ExportHelpers
# Define Saving/Loading Directory and paths:
finalized_output_cache_file='data/pipeline_cache_store.h5'

In [ ]:
# List existing keys in the file:
out_keys = get_h5_data_keys(finalized_output_cache_file=finalized_output_cache_file)
print(out_keys)

In [ ]:
active_config_name = 'maze1'
# active_config_name = 'maze'

desired_spikes_df_key = f'/filtered_sessions/{active_config_name}/spikes_df'
desired_positions_df_key = f'/filtered_sessions/{active_config_name}/pos_df'
# desired_spikes_df_key = f'/filtered_sessions/{active_config_name}/spikes_df'

# Get relevant variables:
# curr_active_pipeline is set above, and usable here
sess: DataSession = curr_active_pipeline.filtered_sessions[active_config_name]
active_computed_data = curr_active_pipeline.computation_results[active_config_name].computed_data
print(f'active_computed_data.keys(): {active_computed_data.keys()}')
pf = curr_active_pipeline.computation_results[active_config_name].computed_data['pf1D']
active_one_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data['pf2D_Decoder']
active_two_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_TwoStepDecoder', None)
active_measured_positions = curr_active_pipeline.computation_results[active_config_name].sess.position.to_dataframe()

In [ ]:
# desired_positions_df_key

output_save_result = save_some_pipeline_data_to_h5(curr_active_pipeline, finalized_output_cache_file=finalized_output_cache_file)
output_save_result

In [ ]:
finalized_output_cache_file = _test_save_pipeline_data_to_h5(curr_active_pipeline, finalized_output_cache_file=finalized_output_cache_file, enable_dry_run=False, enable_debug_print=True)
finalized_output_cache_file

In [ ]:
# Load the saved .h5 spikes dataframe for testing:
desired_spikes_df_key = f'/filtered_sessions/{active_config_name}/spikes_df'
desired_positions_df_key = f'/filtered_sessions/{active_config_name}/pos_df'    
spikes_df, pos_df = load_pipeline_data_from_h5(finalized_output_cache_file=finalized_output_cache_file, desired_spikes_df_key=desired_spikes_df_key, desired_positions_df_key=desired_positions_df_key)


In [ ]:
print(f't_start: {curr_active_pipeline.sess.epochs.t_start}')
print(f't_stop: {curr_active_pipeline.sess.epochs.t_stop}')
# curr_active_pipeline.sess.epochs.t_stop

In [ ]:
curr_sess.epochs.t_start
print(f't_start: {sess.epochs.t_start}')
print(f't_stop: {sess.epochs.t_stop}')

## Test saving out custom computation result:

In [ ]:
# computation_results[active_config_name].computed_data
computation_cache_identifier_key = f'computation_results/{active_config_name}/computed_data'
computation_cache_key = 'burst_detection/burst_intervals'
# active_burst_intervals

filtered_burst_intervals = active_burst_intervals
# Rebuild a single dataframe from: .groupby('aclu').get_group(a_cell_id)
for (a_cell_id, curr_pyburst_interval_df) in active_burst_intervals.items():
    # loop through the cell_ids  
    # Filter to only zero- and first-order bursts:
    # curr_pyburst_interval_df = curr_pyburst_interval_df[curr_pyburst_interval_df['burst_level'] < 2]
    # add the 'aclu' column
    # curr_pyburst_interval_df['aclu'] = a_cell_id
    filtered_burst_intervals[a_cell_id] = curr_pyburst_interval_df

## Concatinate the separate dataframes for each neuron into a single dataframe with an 'aclu' column so that it can be cached/saved to disk:
# pd.concat([s1, s2], keys=['s1', 's2'])
filtered_combined_df = pd.concat(filtered_burst_intervals.values(), keys=list(filtered_burst_intervals.keys()), names=['aclu', 'cell_burst_id'])
# pd.concat(filtered_burst_intervals.values(), keys=list(filtered_burst_intervals.keys()), names=['aclu', 'cell_burst_id'], ignore_index=True)
print_dataframe_memory_usage(filtered_combined_df)
# filtered_combined_df

# Define Saving/Loading Directory and paths:
finalized_computation_cache_file='data/computation_results_cache_store.h5'
computation_cache_identifier_key = f'computation_results/{active_config_name}/computed_data'
filtered_combined_df.to_hdf(finalized_computation_cache_file, key=f'{computation_cache_identifier_key}/{computation_cache_key}')

In [ ]:
# _test_computation_result_output_path = Path('output', active_data_mode_name, sess.name) # active_data_mode_name: 'kdiba'; sess.name: '2006-6-07_11-26-53'
_test_computation_result_output_path = Path('output') # active_data_mode_name: 'kdiba'; sess.name: '2006-6-07_11-26-53'
finalized_computation_results_out_file=_test_computation_result_output_path.joinpath('computation_results_store.h5')
# Do particular key:
computation_result_recording_session_identifier_path = f'{active_data_mode_name}/{sess.name}' # 'kdiba/2006-6-07_11-26-53'

## Specific Config:
active_config_name = 'maze1'
# active_config_name = 'maze2'
# active_config_name = 'maze'
computation_result_identifier_path = f'{computation_result_recording_session_identifier_path}/computation_results/{active_config_name}' # 'kdiba/2006-6-07_11-26-53/computation_results/maze1'
computation_result_identifier_computed_data_path = f'{computation_result_identifier_path}/computed_data' # 'kdiba/2006-6-07_11-26-53/computation_results/maze1/computed_data'
# computation_result_key = 'burst_detection/burst_intervals'

print(f'computation_result_identifier_path: "{computation_result_identifier_path}"')

# Get relevant variables:
# curr_active_pipeline is set above, and usable here
sess: DataSession = curr_active_pipeline.filtered_sessions[active_config_name]

active_computation_results = curr_active_pipeline.computation_results[active_config_name]
active_computed_data = curr_active_pipeline.computation_results[active_config_name].computed_data
active_computation_config = curr_active_pipeline.computation_results[active_config_name].computation_config
active_computation_errors = curr_active_pipeline.computation_results[active_config_name].accumulated_errors
print(f'active_computed_data.keys(): {list(active_computed_data.keys())}')
# print(f'active_computation_errors: {active_computation_errors}')
active_pf_1D = curr_active_pipeline.computation_results[active_config_name].computed_data['pf1D']
active_pf_2D = curr_active_pipeline.computation_results[active_config_name].computed_data['pf2D']    
active_pf_1D_dt = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf1D_dt', None)
active_pf_2D_dt = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_dt', None)
active_one_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_Decoder', None)
active_two_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_TwoStepDecoder', None)
active_eloy_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('EloyAnalysis', None)
active_simpler_pf_densities_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('SimplerNeuronMeetingThresholdFiringAnalysis', None)
active_ratemap_peaks_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', None)
active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
active_measured_positions = curr_active_pipeline.computation_results[active_config_name].sess.position.to_dataframe()
curr_spikes_df = sess.spikes_df

curr_active_config = curr_active_pipeline.active_configs[active_config_name]
curr_active_display_config = curr_active_config.plotting_config

In [ ]:
## Test special pickle
import pickle
# Its important to use binary mode
_test_computation_result_output_path = Path('output') # active_data_mode_name: 'kdiba'; sess.name: '2006-6-07_11-26-53'
finalized_computation_results_out_pickle_test_file=_test_computation_result_output_path.joinpath('examplePickle.pkl')

def saveData(pkl_path, db):
    with open(pkl_path, 'ab') as dbfile:
        # source, destination
        pickle.dump(db, dbfile)                     
        dbfile.close()

def loadData(pkl_path, debug_print=False):
    # for reading also binary mode is important
    db = None
    with open(pkl_path, 'rb') as dbfile:
        db = pickle.load(dbfile)
        if debug_print:
            for keys in db:
                print(keys, '=>', db[keys])
        dbfile.close()
        return db
    return db

In [ ]:
saveData(finalized_computation_results_out_pickle_test_file, db=active_computed_data)
# saveData(finalized_computation_results_out_pickle_test_file, db={'pf2D': active_pf_2D, 'RatemapPeaksAnalysis': active_ratemap_peaks_analysis})

In [ ]:
db = loadData(finalized_computation_results_out_pickle_test_file, debug_print=False)
db

In [ ]:
active_pf_1D.str_for_filename() # 'pf1D-speedThresh_10.00-gridBin_4.00-smooth_2.50-frateThresh_0.20'
active_pf_1D.to_pickle()


In [ ]:
## Note that pickling the whole spikes_df dataframe when done is very quick, especially to load!
active_file_suffix = '.spikes_df_full.pkl'
spikes_df_save_path = curr_active_pipeline.sess.filePrefix.with_suffix(active_file_suffix)
print(f'spikes_df_save_path: {spikes_df_save_path}')

In [ ]:
curr_active_pipeline.sess.spikes_df.to_pickle(spikes_df_save_path)  # where to save it, usually as a .pkl

In [ ]:
loaded_spikes_df = pd.read_pickle(spikes_df_save_path)
loaded_spikes_df

## Test building interactive export Parameters tree GUI to select computation results to save out :

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Params.ParameterTrees.ExportPipelineParametersTree import build_export_parameters_tree
## Build the actual ParameterTree widget, the core GUI
title = 'ExportParamsTest'
app = pg.mkQApp(title)
p = build_export_parameters_tree(curr_active_pipeline, parameter_names='ExportParams', finalized_output_cache_file='data/pipeline_cache_store.h5', include_state_save_restore_buttons=False, debug_print=True)

paramTree = ParameterTree()
paramTree.setParameters(p, showTop=False)
paramTree.show()
paramTree.setWindowTitle(f'PhoParamTreeApp: pyqtgraph ParameterTree: {title}')
paramTree.resize(800,600)

# Testing burst detection: Calculate the spiking profile using pyspike and display the outputs:

### pybursts

In [ ]:
from pybursts import pybursts
import matplotlib.pyplot as plt
from indexed import IndexedOrderedDict
from pyphocorehelpers.print_helpers import print_dataframe_memory_usage
from copy import deepcopy

active_burst_info = active_computed_data['burst_detection']
active_burst_intervals = deepcopy(active_burst_info['burst_intervals'])
filtered_burst_intervals = active_burst_intervals

In [ ]:
# bursting_interval_datasource = Render2DEventRectanglesHelper.build_interval_datasource_from_active_burst_intervals(active_burst_intervals=filtered_burst_intervals)
bursting_interval_datasource = Render2DEventRectanglesHelper.build_burst_event_rectangle_datasource(spike_raster_window.spike_raster_plt_2d, active_burst_intervals, datasource_name='active_burst_intervals_datasource', included_burst_levels=[1,2,3,4,5])
# bursting_interval_datasource

In [ ]:
active_bursts_interval_rects_item = Render2DEventRectanglesHelper.build_IntervalRectsItem_from_interval_datasource(bursting_interval_datasource)
active_bursts_interval_rects_item.setToolTip('Bursting')
active_bursts_interval_rects_item

## Plot the burst periods as rectangles on the 2D Raster Plot:

In [ ]:
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DEventRectanglesHelper import Render2DEventRectanglesHelper

# rect_return_dict = Render2DEventRectanglesHelper.add_event_rectangles(spike_raster_window.spike_raster_plt_2d, active_burst_intervals, included_burst_levels=[1]) # {'interval_rects_item': active_interval_rects_item}
rect_return_dict = Render2DEventRectanglesHelper.add_event_rectangles(spike_raster_window.spike_raster_plt_2d, active_burst_intervals, included_burst_levels=[1,2,3,4,5]) # {'interval_rects_item': active_interval_rects_item}

active_interval_rects_item = rect_return_dict['interval_rects_item']

#### Plot the result:

In [ ]:
main_plot_widget = spike_raster_window.spike_raster_plt_2d.plots.main_plot_widget # PlotItem
background_static_scroll_plot_widget = spike_raster_window.spike_raster_plt_2d.plots.background_static_scroll_window_plot # PlotItem

In [ ]:
## Remove the active_interval_rects_item:
main_plot_widget.removeItem(active_interval_rects_item)
active_interval_rects_item = None

In [ ]:
## Add the active_interval_rects_item to the background_static_scroll_window_plot: 
background_static_scroll_plot_widget.addItem(active_interval_rects_item)

In [ ]:
## Remove the active_interval_rects_item:
background_static_scroll_plot_widget.removeItem(active_interval_rects_item)
active_interval_rects_item = None

## Scratch

In [ ]:
active_burst_intervals_neuron_IDXs_list = spike_raster_window.spike_raster_plt_2d.find_neuron_IDXs_from_cell_ids(list(filtered_burst_intervals.keys()))
# { spike_raster_window.spike_raster_plt_2d.get_neuron_id_and_idx(neuron_id=a_neuron_id)[0] for i, a_neuron_id in enumerate(list(filtered_burst_intervals.keys()))}
neuron_id_qcolors_map = {a_neuron_id:spike_raster_window.spike_raster_plt_2d.params.neuron_qcolors_map[spike_raster_window.spike_raster_plt_2d.get_neuron_id_and_idx(neuron_id=a_neuron_id)[0]] for i, a_neuron_id in enumerate(list(filtered_burst_intervals.keys()))}
neuron_id_qcolors_map

In [ ]:
spike_raster_window.spike_raster_plt_2d.config_fragile_linear_neuron_IDX_map

In [ ]:
spike_raster_window.spike_raster_plt_2d.params.config_items

In [ ]:
print_keys_if_possible('spike_raster_window.spike_raster_plt_2d', spike_raster_window.spike_raster_plt_2d, max_depth=2) 

In [ ]:
print_keys_if_possible('spike_raster_window.spike_raster_plt_2d.params', spike_raster_window.spike_raster_plt_2d.params, max_depth=2) 

In [ ]:
spike_raster_window.spike_raster_plt_2d.plots.scatter_plot

In [ ]:
plt = pg.plot()
plt.addItem(item)
plt.setWindowTitle('pyqtgraph example: customGraphicsItem')

#### Matplotlib broken_barh version:

In [ ]:
y_range = [(i, 1) for i in np.arange(len(curr_pyburst_interval_df.interval_pair))]
print(f'y_range: {y_range}')

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True, constrained_layout=True)

ax0.eventplot(curr_spike_train)
ax0.set_title('Input Timeseries')

# Output intervals:
ax1.set_title('Output Intervals')

ax1.broken_barh(curr_pyburst_interval_df.interval_pair.to_numpy(), (0, 1), facecolors='tab:blue')
# ax1.broken_barh([interval_pairs[1]], (1, 1))
# ax1.broken_barh([interval_pairs[2]], (2, 1))

### Pyspike

In [ ]:
isi_profile = spk.isi_profile(out_pyspike_spiketrains[2], out_pyspike_spiketrains[3])
x, y = isi_profile.get_plottable_data()
plt.plot(x, y, '--k')
print("ISI distance: %.8f" % isi_profile.avrg())
plt.show()


# Can specify the time intervals over which to look at the average ISI:
# isi1 = isi_profile.avrg(interval=(0, 1000))
# isi2 = isi_profile.avrg(interval=(1000, 2000))
# isi3 = isi_profile.avrg(interval=[(0, 1000), (2000, 3000)])
# isi4 = isi_profile.avrg(interval=[(1000, 2000), (3000, 4000)])

# Bivariate (across only two spiketrains, specified here as [2] and [3]) version:
spike_profile = spk.spike_profile(out_pyspike_spiketrains[2], out_pyspike_spiketrains[3])
x, y = spike_profile.get_plottable_data()
plt.plot(x, y, '--k')
print("SPIKE distance: %.8f" % spike_profile.avrg())
plt.show()


## Multivariate (across all spike trains at at time) version:
spiketrains_list = list(out_pyspike_spiketrains.values())
avrg_isi_profile = spk.isi_profile(spiketrains_list)
avrg_spike_profile = spk.spike_profile(spiketrains_list)
avrg_spike_sync_profile = spk.spike_sync_profile(spiketrains_list)

In [ ]:
def _compute_distance_matricies(spike_trains):
    plt.figure()
    isi_distance_mat = spk.isi_distance_matrix(spike_trains)
    plt.imshow(isi_distance_mat, interpolation='none')
    plt.title("ISI-distance")

    plt.figure()
    spike_distance_mat = spk.spike_distance_matrix(spike_trains) # , interval=(0.001,1000.0)
    plt.imshow(spike_distance_mat, interpolation='none')
    plt.title("SPIKE-distance")

    plt.figure()
    spike_sync_mat = spk.spike_sync_matrix(spike_trains) # , interval=(2000.0,4000.0)
    plt.imshow(spike_sync_mat, interpolation='none')
    plt.title("SPIKE-Sync")

    plt.show()
    return isi_distance_mat, spike_distance_mat, spike_sync_mat
    

isi_distance_mat, spike_distance_mat, spike_sync_mat = _compute_distance_matricies(spiketrains_list)

In [ ]:
x, y = avrg_spike_profile.get_plottable_data()
plt.plot(x, y, 'k')
print("SPIKE distance: %.8f" % spike_profile.avrg())
plt.show()

In [ ]:
# 
x, y = avrg_isi_profile.get_plottable_data()
plt.plot(x, y, '--k')
# print("ISI distance: %.8f" % isi_profile.avrg())
plt.show()

In [ ]:
from vedo import *
from vedo.pyplot import violin, whisker, histogram

## Example Violin plot with 3 generated violins:
# n = 1000
# acts = [
#     Text3D('gaussian', pos=(0,4.5), s=0.3, c='k', justify='center'),
#     violin(np.random.randn(n)),

#     Text3D('exponential', pos=(5,-1), s=0.3, c='k', justify='center'),
#     violin(np.random.exponential(1, n), x=5, width=3, spline=False, centerline=False, c='t', lc='k'),

#     Text3D('chisquare', pos=(10,11), s=0.3, c='k', justify='center'),
#     violin(np.random.chisquare(9, n)/4, x=10, vlim=(0,10), c='lg', lc='dg'),
# ]


## Actual violin plot from each cell's spike ISIs:
intra_unit_spacing = 10.0

# axes_options_dict = dict(xtitle=False, ytitle='distribution')
axes_options_dict = dict(xtitle=False, ytitle=False)

acts = []
for (i, a_cell_id) in enumerate(curr_active_pipeline.sess.spikes_df.spikes.neuron_ids):
    if i < 3:
        # loop through the cell_ids
        curr_unit_x_offset = float(i) * intra_unit_spacing
        curr_df = curr_active_pipeline.sess.spikes_df.groupby('aclu').get_group(a_cell_id)
        # curr_active_pipeline.sess.spikes_df['scISI']
        # ValueError: autodetected range of [nan, nan] is not finite
        # curr_v_text = Text3D(f'unit[{a_cell_id}]', pos=(curr_unit_x_offset,-1), s=0.3, c='k', justify='center')
        # acts.append(curr_v_text)
        # curr_v = violin(curr_df['scISI'].dropna(), bins=20, x=curr_unit_x_offset, centerline=True, spline=False, vlim=[0, 10]) # width=20.0, 
        curr_v = whisker(curr_df['scISI'].dropna(), s=0.25, c='k', lw=2, bc='blue', alpha=0.25, r=5, jitter=True, horizontal=False)
        # curr_v = histogram(curr_df['scISI'].dropna(), errors=True, alpha=0.8, logscale=True)
        # curr_v = histogram(curr_df['scISI'].dropna(), errors=True, alpha=0.8, logscale=False)
        # curr_v = violin(curr_df['scISI'].dropna())
        acts.append(curr_v)


show(acts, axes=axes_options_dict).close()

## TODO: comes see https://github.com/marcomusy/vedo/blob/master/examples/pyplot/whiskers.py to implement a whisker's plot from this data.

In [ ]:
# note that this is very slow, but works
included_cell_INDEXES = np.array([spike_raster_plt.get_neuron_id_and_idx(neuron_id=an_included_cell_ID)[0] for an_included_cell_ID in spike_raster_plt.spikes_df['aclu'].to_numpy()]) # get the indexes from the cellIDs
spike_raster_plt.spikes_df['cell_idx'] = included_cell_INDEXES.copy()

In [ ]:
# spike_raster_plt.spikes_df
np.unique(spike_raster_plt.spikes_df['cell_idx']) # array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39])

In [ ]:
# out_figs = curr_active_pipeline.sess.spikes_df.plot.hist(by='aclu', column='scISI', bins=12, sharex=True, alpha=0.5, figsize=(40, 120))
# out_figs = curr_active_pipeline.sess.spikes_df.plot.hist(by='aclu', column='scISI', bins=12, alpha=0.5, figsize=(40, 120))
# out_figs = curr_active_pipeline.sess.spikes_df.plot.hist(column='scISI', bins=12, alpha=0.5, figsize=(40, 120))
# out_figs = curr_active_pipeline.sess.spikes_df.plot.box(column=['scISI'], by='aclu', figsize=(40, 120)) # boxplots side-by-side

# out_figs = curr_active_pipeline.sess.spikes_df.plot.kde(column=['scISI'], by='aclu', figsize=(40, 120)) # Not working
# out_figs = curr_active_pipeline.sess.spikes_df.plot(column=['scISI'], by='aclu', figsize=(40, 120)) # boxplots side-by-side

# out_figs = curr_active_pipeline.sess.spikes_df.plot(kind='hist', sharex=True, sharey=False, column=['scISI'], by='aclu', figsize=(40, 120)) # boxplots side-by-side
out_figs = curr_active_pipeline.sess.spikes_df.plot(kind='scatter', sharex=True, sharey=False, column=['scISI'], by='aclu', figsize=(40, 120)) # boxplots side-by-side
# 

# .plot(xlabel="new x", ylabel="new y");


In [ ]:
from PendingNotebookCode import _perform_PBE_stats

pbe_analyses_result_df, [all_epochs_full_pbe_spiketrain_lists, all_epochs_pbe_num_spikes_lists, all_epochs_intra_pbe_interval_lists] = _perform_PBE_stats(curr_active_pipeline, debug_print=False) # all_epochs_n_pbes: [206, 31, 237], all_epochs_mean_pbe_durations: [0.2209951456310722, 0.23900000000001073, 0.22335021097046923], all_epochs_cummulative_pbe_durations: [45.52500000000087, 7.409000000000333, 52.934000000001205], all_epochs_total_durations: [1716.8933641185379, 193.26664069312392, 1910.1600048116618]
pbe_analyses_result_df
# pbe_analyses_result_df.to_clipboard(sep=',') 

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_SimplePlot import plot_simple_graph

[p1], win, app = plot_simple_graph(all_epochs_intra_pbe_interval_lists[0])

for (idx, named_range) in enumerate(curr_named_timeranges):
    # interested in analyzing both the filtered_sess.pbe and the filtered_sess.spikes_df (as they relate to the PBEs)
    p1.plot(all_epochs_intra_pbe_interval_lists[idx] + (300 * idx), pen=(255,0,0), name=named_range.name)

# p1.plot(np.random.normal(size=100), pen=(255,0,0), name="Red curve")
# p1.plot(np.random.normal(size=110)+5, pen=(0,255,0), name="Green curve")
# p1.plot(np.random.normal(size=120)+10, pen=(0,0,255), name="Blue curve")

# def plot_simple_graph(y=np.random.normal(size=100))
#     app = pg.mkQApp("Plotting Example")
#     #mw = QtGui.QMainWindow()
#     #mw.resize(800,800)
#     
#     win = pg.GraphicsLayoutWidget(show=True, title="Basic plotting examples")
#     win.resize(1000,600)
#     win.setWindowTitle('pyqtgraph example: Plotting')
#     
#     # Enable antialiasing for prettier plots
#     pg.setConfigOptions(antialias=True)
#     
#     p1 = win.addPlot(title="Basic array plotting", y=y)
#     
#     return app, win, [p1]

In [ ]:
# Find all spikes that occur during both a PBE & a lap on the track:

# curr_active_pipeline.sess.spikes_df[(curr_active_pipeline.sess.spikes_df['PBE_id'] > -1)] # & (curr_active_pipeline.sess.spikes_df['lap'] != -1)
during_lap_and_PBE_spikes_df = curr_active_pipeline.sess.spikes_df[(curr_active_pipeline.sess.spikes_df['PBE_id'] > -1) & (curr_active_pipeline.sess.spikes_df['lap'] != -1)]
# curr_active_pipeline.sess.spikes_df[(curr_active_pipeline.sess.spikes_df['lap'] > -1)]
during_lap_and_PBE_spikes_df

# updated_spikes_df = curr_active_pipeline.sess.compute_PBEs_spikes_df(curr_active_pipeline.sess.spikes_df, curr_active_pipeline.sess.pbe.to_dataframe())
# updated_spikes_df[(updated_spikes_df['PBE_id'] > -1)]
# curr_active_pipeline.sess.spikes_df[(curr_active_pipeline.sess.spikes_df['PBE_id'] > -1)]
# curr_active_pipeline.sess.spikes_df

# group by Team, get mean, min, and max value of Age for each value of Team.
grouped_single = during_lap_and_PBE_spikes_df.groupby('lap').agg({'PBE_id': ['mean', 'min', 'max']})
grouped_single = grouped_single.reset_index()
print(grouped_single)

In [ ]:
## Find rects for epochs in the active window:

from pyphocorehelpers.geometry_helpers import find_ranges_in_window
    
# is_range_in_window, included_epoch_indicies, included_epoch_starts, included_epoch_ends, included_epoch_is_truncated = find_ranges_in_window(curr_sess.pbe.starts, curr_sess.pbe.stops, (595.0, 820.0))

is_range_in_window, included_epoch_indicies, included_epoch_starts, included_epoch_ends, included_epoch_is_truncated = find_ranges_in_window(curr_sess.pbe.starts, curr_sess.pbe.stops, (598.65, 820.0))

# 'is_range_in_window': is_range_in_window
pd.DataFrame({'included_epoch_indicies':included_epoch_indicies, 'included_epoch_starts':included_epoch_starts, 'included_epoch_ends':included_epoch_ends, 'included_epoch_is_truncated': included_epoch_is_truncated})

# print(f'included_epoch_indicies: {np.shape(included_epoch_indicies)}')
# print(f'included_epoch_starts: {np.shape(included_epoch_starts)}')
# pd.DataFrame({'included_epoch_starts':included_epoch_starts, 'included_epoch_ends':included_epoch_ends, 'included_epoch_is_truncated': included_epoch_is_truncated})

# included_epoch_indicies
# included_epoch_is_truncated

In [ ]:
spike_raster_plt_3d.ui.spinRenderWindowDuration.setValue(scroll_window_width)

In [ ]:
spike_raster_plt_3d.ui.spinTemporalZoomFactor.setValue(10.0)

In [ ]:
spike_raster_plt.animation_time_step # 0.03 (seconds)

In [ ]:
# curr_transform = spike_raster_plt.ui.gz.transform() # PyQt5.QtGui.QMatrix4x4(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, -10.0, 0.0, 0.0, 0.0, 1.0)
curr_transform = spike_raster_plt.ui.gx.transform() # PyQt5.QtGui.QMatrix4x4(0.0, 0.0, 1.0, -20.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)
print(curr_transform)

curr_transform.matrix()
# spike_raster_plt.ui.gx.

In [ ]:
spike_raster_plt.params.keys()

In [ ]:
# spike_raster_plt.params.spike_start_z # -10
spike_raster_plt.params.spike_end_z = -8.0 # -6.0

In [ ]:
spike_raster_plt.animation_time_step = 0.06

In [ ]:
spike_raster_plt.shift_animation_frame_val(-3)


# Testing: Position Decoding:

In [ ]:
# win.close()

# np.set_printoptions(edgeitems=5, linewidth=100000, formatter=dict(float=lambda x: "%g" % x))

np.set_string_function(lambda x:f'np.array[{np.shape(x)}]', repr=True)
# with np.printoptions(precision=2, edgeitems=2, linewidth=144):
with np.printoptions(precision=3, edgeitems=2, linewidth=100000):
    # active_computed_data['pf2D'].ratemap
    print(active_computed_data['pf2D'])
    # print(active_computed_data['pf2D'].ratemap)
    
    


np.set_string_function(None) # restore default

In [ ]:
len(active_computed_data['pf2D'].ratemap_spiketrains)

#.tuning_curves

In [ ]:
## Stock Decoder:
from neuropy.analyses.decoders import Decode1d

def stock_1d_decoder(sess, pf, curr_result_label):
    maze1 = sess.paradigm[curr_result_label]
    # rpls = sess.ripple.time_slice(maze1[0], maze1[1])
    rpls = None
    pf_neurons = sess.neurons.get_by_id(pf.ratemap.neuron_ids)
    decode = Decode1d(neurons=pf_neurons, ratemap = pf.ratemap, epochs=rpls, bin_size=0.02)
    return decode

def validate_stock_1d_decoder(sess, decode):
    # Plot to validate decoder:
    np.shape(decode.decoded_position) # (85845,)
    plt.plot(decode.decoded_position)
    ax = plt.gca()
    # ax.xlim() # (-4292.2, 90136.2)
    ax.set_xlim(10000, 12000)

np.shape(decode.posterior) # (48, 85845)

<!-- % $$\int_{a}^b f(x)dx$$ -->
<!-- Euler's identity: $ e^{i \pi} + 1 = 0 $ -->

## One-step Bayesian Decoder:
$$P(\overrightarrow{x}_{t}|\overrightarrow{n}_{t})$$

$$P(\overrightarrow{n}|\overrightarrow{x})$$ : probability for the numbers of spikes $\overrightarrow{n}$ to occur given we know the animal is at location $\overrightarrow{x}$

## Two-step Bayesian Decoder:
$$P(\overrightarrow{x}_{t}|\overrightarrow{n}_{t}, \overrightarrow{x}_{t-1}) = k P(\overrightarrow{x}_{t}|\overrightarrow{n}_{t}) P(\overrightarrow{x}_{t-1}|\overrightarrow{x}_{t})$$

In [ ]:

# active_two_step_decoder['most_likely_positions'].shape # (2, 1717)

active_two_step_decoder['most_likely_position_indicies'].shape # (2, 1717)
np.max(active_two_step_decoder['most_likely_position_indicies'], axis=1) # array([0, 1])
active_two_step_decoder['most_likely_position_indicies']
# active_two_step_decoder['p_x_given_n_and_x_prev'].shape # (59, 21, 1717)

# np.nanmax(active_two_step_decoder['p_x_given_n_and_x_prev'], axis=(1, 2)) # (59,)
# np.nanmax(active_two_step_decoder['p_x_given_n_and_x_prev'], axis=-1).shape # (59, 21)

# np.nanmax(active_two_step_decoder['p_x_given_n_and_x_prev'], axis=-1)
# np.max(active_two_step_decoder['most_likely_positions'], axis=1) # array([ 36.30101033, 128.49991842])


# np.max(active_one_step_decoder.most_likely_positions, axis=0) # array([244.02731273, 148.3231301 ])

## PyQtPlot Exploration

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import pyqtplot_plot_image_array, pyqtplot_plot_image

# test single image plot:
curr_im = np.squeeze(active_one_step_decoder.ratemap.normalized_tuning_curves[0,:,:]) # (43, 63, 63)
app, win, imv = pyqtplot_plot_image(active_one_step_decoder.xbin, active_one_step_decoder.ybin, curr_im)
win.show()

In [ ]:
pg.mkQApp()
# Create ScatterPlotWidget and configure its fields
spw = pg.ScatterPlotWidget()
# spw.setFields([
    # ('x_pos', {'units': 'm'}),
#     ('y_pos', {'units': 'm'}),
#     ('count', {}),
#     ('amplitude', {'units': 'V'}),
#     ('decay', {'units': 's'}),    
#     ('type', {'mode': 'enum', 'values': strings}),
#     ])

spw.setFields([
    ('x', {'units': 'm'}),
    ('y', {'units': 'm'}),
    ('lin_pos', {'units': 'm'}),
    ('speed', {'units': 'm/s'}),
    ('binned_x', {}),
    ('binned_y', {}),
    # ('type', {'mode': 'enum', 'values': strings}),
])
    
spw.setData(time_binned_pos_df)
spw.show()


# ## Multiple Line Plots:
# plotWidget = pg.plot(title='PhoTest PyQtPlot Widget')
# for i in range(3):
#     plotWidget.plot(x, y[i], pen=(i,3))
    

### Custom DataTree Widget that displays a nested hierarchy of data:

In [ ]:
# d = {
#     'active_sess_config':curr_active_pipeline.active_sess_config.__dict__,
#     'active_configs':curr_active_pipeline.active_configs,
#     'active_session_computation_configs':active_session_computation_configs[0].__dict__
# }
d = {
    "computed_data": curr_active_pipeline.computation_results['maze1'].computed_data,
    # "computation_results['maze1']'": curr_active_pipeline.computation_results['maze1'],
}
# d = {
#     'active_session_computation_configs':active_session_computation_configs,
#     'active_two_step_decoder': active_two_step_decoder,
#     'active_extended_stats': active_extended_stats
# }

from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_DataTreeWidget import plot_dataTreeWidget
tree, app = plot_dataTreeWidget(data=d, title='PhoOutputDataTreeApp')
tree.show()

In [ ]:
# tree.show()
tree = None
app = None

In [ ]:
tree.resize(1920, 1200) # pyqtgraph.widgets.DataTreeWidget.DataTreeWidget
tree.resizeColumnToContents(2)
# setFormat(format, column=None)
# tree.resizeColumnToContents(1)
# tree.expandToDepth
print(tree.dumpObjectInfo())
# tree.expandAll()
# tree.nodes # a dict

# tree.hideColumn(2) # hides the value column
# print(tree.children()[0].dumpObjectInfo())
# # tree.children()[1] # <PyQt5.QtWidgets.QStyledItemDelegate at 0x197b467fca0>
# # tree.children()[2]
# # tree.children()[3].dumpObjectTree() # QAbstractItemModel
# print(tree.children()[3].dumpObjectInfo())
# # listAllItems()
# # tree.itemFromIndex(0)
# # tree.findChild()
# # listAllItems()
# tree.collapseAll()

In [ ]:
# InteractivePlaceCellConfig
from pprint import pprint
pprint(curr_active_pipeline.active_configs['maze1'].__dict__)


In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Params.pyqtplot_ParamTreeWidget import plot_paramTreeWidget
param_tree, param_tree_app = plot_paramTreeWidget(title='PhoMainParamTreeApp')

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Flowchart.pyqtplot_Flowchart import plot_flowchartWidget
pipeline_flowchart_window, pipeline_flowchart_app = plot_flowchartWidget(title='PhoMainPipelineFlowchartApp')

## Check Placefield Normalizations:
Conclusion: neither the normalized_tuning_curves nor tuning_curves are normalized in any way! They give different firing rates across time.
NOTE: For the pyramidal-only and lap-epoch filtered Diba data, the np.nanmax of normalized_tuning_curves actually does appear to be scaled to a maximum of 1.0 across all units, meaning only the relative difference between units in firing rate is preserved.

In [ ]:
active_one_step_decoder.pf.ratemap

In [ ]:
np.sum(active_one_step_decoder.pf.ratemap.normalized_tuning_curves, axis=(1,2)) # ERROR: the normalized_tuning_curves are NOT normalized in any way!

In [ ]:
np.sum(active_one_step_decoder.pf.ratemap.normalized_tuning_curves, axis=(1,2)) # ERROR: the normalized_tuning_curves are NOT normalized in any way!
np.nanmax(active_one_step_decoder.pf.ratemap.normalized_tuning_curves, axis=(1,2)) # Not even by having their maximum value scaled to one!

# np.sum(active_one_step_decoder.pf.ratemap.tuning_curves, axis=(1,2))
# np.nanmax(active_one_step_decoder.pf.ratemap.tuning_curves, axis=(1,2))

In [ ]:
active_pf_2D.ratemap.unit_max_tuning_curves

# # test_unit_max_tuning_curve = active_pf_2D.ratemap.tuning_curves[0] / np.max(active_pf_2D.ratemap.tuning_curves[0])

# test_unit_max_tuning_curves = [a_tuning_curve / np.max(a_tuning_curve) for a_tuning_curve in active_pf_2D.ratemap.tuning_curves]
# validate_unit_max = [np.max(a_unit_max_tuning_curve) for a_unit_max_tuning_curve in test_unit_max_tuning_curves]
# # print(f'validate_unit_max: {validate_unit_max}')
# assert np.allclose(validate_unit_max, np.full_like(validate_unit_max, 1.0), equal_nan=True), f"unit_max_tuning_curves doesn't have a max==1.0 after scaling!!! Maximums: {validate_unit_max}"
# np.isclose(
# validate_unit_max
# test_unit_max_tuning_curves
# np.max(test_unit_max_tuning_curve)
# np.min(test_unit_max_tuning_curve)


# np.min(active_pf_2D.ratemap.minmax_normalized_tuning_curves[0])

In [ ]:
np.max(active_pf_2D.ratemap.pdf_normalized_tuning_curves[0])

## Placefield Overlap Detection:

In [ ]:
# Test Placefield Overlap Detection:
def compute_placefield_overlap(pf):
    return np.squeeze(np.prod(pf, axis=0))


active_pf_overlap_results = curr_active_pipeline.computation_results[active_config_name].computed_data['placefield_overlap']
all_pairwise_neuron_IDs_combinations = active_pf_overlap_results['all_pairwise_neuron_IDs_combinations']
total_pairwise_overlaps = active_pf_overlap_results['total_pairwise_overlaps']
all_pairwise_overlaps = active_pf_overlap_results['all_pairwise_overlaps']

print_keys_if_possible('active_pf_overlap_results', active_pf_overlap_results)

In [ ]:
# active_placefield_overlap
# total_pairwise_overlaps
# all_pairwise_overlaps


# top_pairwise_overlaps = all_pairwise_overlaps[0:9,:,:]

top_pairwise_overlaps = np.squeeze(all_pairwise_overlaps[2,:,:])

from pyphoplacecellanalysis.GUI.PyQtPlot.pyqtplot_Matrix import MatrixRenderingWindow
print(f'np.shape(top_pairwise_overlaps): {np.shape(top_pairwise_overlaps)}')
pg.mkQApp("Correlation matrix display")
main_window = MatrixRenderingWindow(matrix=top_pairwise_overlaps, columns=[f'{i}' for i in np.arange(np.shape(top_pairwise_overlaps)[-1])])

# compute_placefield_overlap(active_one_step_decoder.pf.ratemap.normalized_tuning_curves)

# Placefield Renormalization - by area under the curve

In [ ]:
# from PhoPyPlaceCellReplayAnalysis


In [ ]:
# test single image plot:
curr_im = np.squeeze(active_one_step_decoder.ratemap.normalized_tuning_curves[0,:,:]) # (43, 63, 63)
app, win, imv = pyqtplot_plot_image(active_one_step_decoder.xbin, active_one_step_decoder.ybin, curr_im)
win.show()

# Test PfND_TimeDependent Class

## Old TimeSynchronized*Plotter Testing

In [ ]:
t = curr_occupancy_plotter.active_time_dependent_placefields.last_t + 7 # add one second
# with np.errstate(divide='ignore', invalid='ignore'):
# active_time_dependent_placefields.update(t)
print(f't: {t}')
curr_occupancy_plotter.on_window_changed(0.0, t)
curr_placefields_plotter.on_window_changed(0.0, t)

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import pyqtplot_plot_image, pyqtplot_plot_image_array
import time

def _test_plot_curr_pf_result(curr_t, curr_ratemap, drop_below_threshold: float=0.0000001, output_plots_dict=None):
    """ plots a single result at a given time.
    
    Creates the figures if needed, otherwise updates the existing ones.
    
    """
    if output_plots_dict is None:
        output_plots_dict = {'occupancy': {}, 'placefields': {}} # make a new dictionary to hold the plot objects.

    # images = curr_ratemap.tuning_curves # (43, 63, 63)
    occupancy = curr_ratemap.occupancy
    # occupancy = curr_ratemap.curr_raw_occupancy_map

    imv = output_plots_dict.get('occupancy', {}).get('imv', None)
    if imv is None:
        # Otherwise build the plotter:
        occupancy_app, occupancy_win, imv = pyqtplot_plot_image(active_time_dependent_placefields2D.xbin, active_time_dependent_placefields2D.ybin, occupancy)
        output_plots_dict['occupancy'] = dict(zip(('app', 'win', 'imv'), (occupancy_app, occupancy_win, imv)))   
        occupancy_win.show()
    else:
        # Update the existing one:
        imv.setImage(occupancy, xvals=active_time_dependent_placefields2D.xbin)

    pg.QtGui.QApplication.processEvents() # call to ensure the occupancy gets updated before starting the placefield plots:
    
    img_item_array = output_plots_dict.get('placefields', {}).get('img_item_array', None)
    if img_item_array is None:
        # Create a new one:
        placefields_app, placefields_win, root_render_widget, plot_array, img_item_array, other_components_array = pyqtplot_plot_image_array(active_time_dependent_placefields2D.xbin, active_time_dependent_placefields2D.ybin,
                                                                                                                                           active_time_dependent_placefields2D.ratemap.normalized_tuning_curves, active_time_dependent_placefields2D.curr_raw_occupancy_map)#, 
        output_plots_dict['placefields'] = dict(zip(('app', 'win', 'root_render_widget', 'plot_array', 'img_item_array', 'other_components_array'), (placefields_app, placefields_win, root_render_widget, plot_array, img_item_array, other_components_array)))
        placefields_win.show()

    else:
        # Update the placefields plot if needed:
        images = curr_ratemap.tuning_curves # (43, 63, 63)
        for i, an_img_item in enumerate(img_item_array):
            image = np.squeeze(images[i,:,:])
            # Pre-filter the data:
            # image = np.array(image.copy()) / np.nanmax(image) # note scaling by maximum here!
            if drop_below_threshold is not None:
                image[np.where(occupancy < drop_below_threshold)] = np.nan # null out the occupancy        
            # an_img_item.setImage(np.squeeze(images[i,:,:]))
            an_img_item.setImage(image)

    return output_plots_dict

In [ ]:
def pre_build_iterative_results(num_iterations=50, t_list=[], ratemaps_list=[]):
    """ 
    build up historical data arrays:
    
    Usage:
        t_list, ratemaps_list = pre_build_iterative_results(num_iterations=50, t_list=t_list, ratemaps_list=ratemaps_list)
    """
    # t_list = []
    # ratemaps_list = []
    
    def _step_plot(time_step_seconds):
        t = active_time_dependent_placefields2D.last_t + time_step_seconds # add one second
        t_list.append(t)
        with np.errstate(divide='ignore', invalid='ignore'):
            active_time_dependent_placefields2D.update(t)
        # Loop through and update the plots:
        # Get flat list of images:
        curr_ratemap = active_time_dependent_placefields2D.ratemap
        # images = curr_ratemap.tuning_curves # (43, 63, 63)
        # images = active_time_dependent_placefields2D.ratemap.normalized_tuning_curves[0:40,:,:] # (43, 63, 63)
        # occupancy = curr_ratemap.occupancy
        ratemaps_list.append(curr_ratemap)
    #     for i, an_img_item in enumerate(img_item_array):
    #     # for i, a_plot in enumerate(plot_array):
    #         # image = np.squeeze(images[i,:,:])
    #         # Pre-filter the data:
    #         # image = np.array(image.copy()) / np.nanmax(image) # note scaling by maximum here!
    # #         if drop_below_threshold is not None:
    # #             image[np.where(occupancy < drop_below_threshold)] = np.nan # null out the occupancy        
    #         an_img_item.setImage(np.squeeze(images[i,:,:]))
    
    for i in np.arange(num_iterations):
        _step_plot(time_step_seconds=1.0)
    
    return t_list, ratemaps_list

# Loop through the historically collected ratemaps and plot them:
def _test_plot_historical_iterative_pf_results(t_list, ratemaps_list, drop_below_threshold: float=0.0000001, output_plots_dict=None):
    """ Uses the previously built-up t_list and ratemaps_list (as computed by pre_build_iterative_results(...)) to plot the time-dependent results.
    requires:
    imv: a previously created single-image plotter:
    """
    num_historical_results = len(ratemaps_list)
    assert len(t_list) == len(ratemaps_list), f"len(t_list): {len(t_list)} needs to equal len(ratemaps_list): {len(ratemaps_list)}"
    
    if output_plots_dict is None:
        output_plots_dict = {'occupancy': {},
                            'placefields': {}} # make a new dictionary to hold the plot objects.
        
    for i in np.arange(num_historical_results):
        curr_t = t_list[i]
        # Set up
        # print(f'curr_t: {curr_t}')
        curr_ratemap = ratemaps_list[i]
        output_plots_dict = _test_plot_curr_pf_result(curr_t, curr_ratemap, drop_below_threshold=drop_below_threshold, output_plots_dict=output_plots_dict)
    
        pg.QtGui.QApplication.processEvents()
        time.sleep(0.1) # Sleep for 0.5 seconds

    return output_plots_dict

# Build the Historical Results:
t_list, ratemaps_list = pre_build_iterative_results(num_iterations=50, t_list=t_list, ratemaps_list=ratemaps_list)
# Plot the historical results:
if output_plots_dict is None:
    output_plots_dict = {'occupancy': {}, 'placefields': {}}
output_plots_dict = _test_plot_historical_iterative_pf_results(t_list, ratemaps_list, output_plots_dict=output_plots_dict)

In [ ]:
# Compute the time-dependent ratemap info in real-time and plot them:
def _test_step_live_iterative_pf_results_plot(active_time_dependent_placefields2D, t, drop_below_threshold: float=0.0000001, output_plots_dict=None):
    """ 
    requires:
    imv: a previously created single-image plotter:
    """
    # Compute the updated placefields/occupancy for the time t:
    with np.errstate(divide='ignore', invalid='ignore'):
        active_time_dependent_placefields2D.update(t)
    # Update the plots:
    curr_t = active_time_dependent_placefields2D.last_t
    curr_ratemap = active_time_dependent_placefields2D.ratemap

    if output_plots_dict is None:
        output_plots_dict = {'occupancy': {}, 'placefields': {}} # make a new dictionary to hold the plot objects.
        
    # Plot the results directly from the active_time_dependent_placefields2D
    output_plots_dict = _test_plot_curr_pf_result(curr_t, curr_ratemap, drop_below_threshold=drop_below_threshold, output_plots_dict=output_plots_dict)
    pg.QtGui.QApplication.processEvents()
    
    return output_plots_dict

def _test_live_iterative_pf_results_plot(active_time_dependent_placefields2D, num_iterations=50, time_step_seconds=1.0, drop_below_threshold: float=0.0000001, output_plots_dict=None):
    """ performs num_iterations time steps of size time_step_seconds and plots the results. """
    for i in np.arange(num_iterations):
        t = active_time_dependent_placefields2D.last_t + time_step_seconds # add one second
        output_plots_dict = _test_step_live_iterative_pf_results_plot(active_time_dependent_placefields2D, t, drop_below_threshold=drop_below_threshold, output_plots_dict=output_plots_dict)
        time.sleep(0.1) # Sleep for 0.5 seconds

In [ ]:
try:
    if output_plots_dict is None:
        output_plots_dict = {'occupancy': {}, 'placefields': {}}
except NameError:
    output_plots_dict = {'occupancy': {}, 'placefields': {}}

output_plots_dict = _test_live_iterative_pf_results_plot(active_time_dependent_placefields2D, num_iterations=50, time_step_seconds=1.0, output_plots_dict=output_plots_dict)

In [ ]:
output_plots_dict = {'occupancy': {}, 'placefields': {}} # clear the output plots dict
output_plots_dict = _test_step_live_iterative_pf_results_plot(active_time_dependent_placefields2D, spike_raster_window.spikes_window.active_time_window[1], output_plots_dict=output_plots_dict)

In [ ]:
def _on_window_updated(window_start, window_end):
    # print(f'_on_window_updated(window_start: {window_start}, window_end: {window_end})')
    global output_plots_dict
    ## Update only version:
    # with np.errstate(divide='ignore', invalid='ignore'):
    #     active_time_dependent_placefields2D.update(window_end) # advance the placefield display to the end of the window.
    ## Update and plot version:
    # t = window_end
    output_plots_dict = _test_step_live_iterative_pf_results_plot(active_time_dependent_placefields2D, window_end, output_plots_dict=output_plots_dict)
    
# spike_raster_window.connect_additional_controlled_plotter(_on_window_updated)

_on_window_updated(spike_raster_window.spikes_window.active_time_window[0], spike_raster_window.spikes_window.active_time_window[1])
sync_connection = spike_raster_window.spike_raster_plt_2d.window_scrolled.connect(_on_window_updated) # connect the window_scrolled event to the _on_window_updated function

In [ ]:
active_time_dependent_placefields2D.plot_occupancy()

In [ ]:
# active_time_dependent_placefields2D.plot_ratemaps_2D(enable_spike_overlay=False) # Works
active_time_dependent_placefields2D.plot_ratemaps_2D(enable_spike_overlay=True)

In [ ]:
# t_list
active_time_dependent_placefields2D.plot_ratemaps_2D(enable_saving_to_disk=False, enable_spike_overlay=False)

In [ ]:
# ax_pf_1D, occupancy_fig, active_pf_2D_figures, active_pf_2D_gs = plot_all_placefields(None, active_time_dependent_placefields2D, active_config_name)
occupancy_fig, occupancy_ax = active_time_dependent_placefields2D.plot_occupancy(identifier_details_list=[])

In [ ]:
i = 0
while (i < len(t_list)):
    curr_t = t_list[i]
    # Set up
    print(f'curr_t: {curr_t}')
    curr_ratemap = ratemaps_list[i]
    # images = curr_ratemap.tuning_curves # (43, 63, 63)
    occupancy = curr_ratemap.occupancy
    # occupancy = curr_ratemap.curr_raw_occupancy_map
    imv.setImage(occupancy, xvals=active_time_dependent_placefields2D.xbin)
    i += 1
    pg.QtGui.QApplication.processEvents()
    
print(f'done!')

In [ ]:
# Timer Update Approach:
timer = pg.QtCore.QTimer()
i = 0
def update():
    if (i < len(t_list)):
        curr_t = t_list[i]
        # Set up
        print(f'curr_t: {curr_t}')
        curr_ratemap = ratemaps_list[i]
        # images = curr_ratemap.tuning_curves # (43, 63, 63)
        occupancy = curr_ratemap.occupancy
        # occupancy = curr_ratemap.curr_raw_occupancy_map
        imv.setImage(occupancy, xvals=active_time_dependent_placefields2D.xbin)
        i += 1
    else:
        print(f'done!')
    # pw.plot(x, y, clear=True)

timer.timeout.connect(update)

In [ ]:
# timer.start(16)
timer.start(500)


In [ ]:
timer.stop()

In [ ]:
t_list

In [ ]:
# get properties from spike_raster_window:

active_curve_plotter_3d = test_independent_vedo_raster_widget # use separate vedo plotter
# active_curve_plotter_3d = spike_raster_window.spike_raster_plt_3d
curr_computations_results = curr_active_pipeline.computation_results[active_config_name]

In [ ]:
## Spike Smoothed Moving Average Rate:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.TimeCurves3D.Specific3DTimeCurves import Specific3DTimeCurvesHelper
binned_spike_moving_average_rate_curve_datasource = Specific3DTimeCurvesHelper.add_unit_time_binned_spike_visualization_curves(curr_computations_results, active_curve_plotter_3d, spike_visualization_mode='mov_average')            

In [ ]:
# Get current plot items:
curr_plot3D_active_window_data = active_curve_plotter_3d.params.time_curves_datasource.get_updated_data_window(active_curve_plotter_3d.spikes_window.active_window_start_time, active_curve_plotter_3d.spikes_window.active_window_end_time) # get updated data for the active window from the datasource
is_data_series_mode = active_curve_plotter_3d.params.time_curves_datasource.has_data_series_specs
if is_data_series_mode:
    data_series_spaital_values_list = active_curve_plotter_3d.params.time_curves_datasource.data_series_specs.get_data_series_spatial_values(curr_plot3D_active_window_data)
    num_data_series = len(data_series_spaital_values_list)

curr_data_series_index = 0
curr_data_series_dict = data_series_spaital_values_list[curr_data_series_index]

curr_plot_column_name = curr_data_series_dict.get('name', f'series[{curr_data_series_index}]') # get either the specified name or the generic 'series[i]' name otherwise
curr_plot_name = active_curve_plotter_3d.params.time_curves_datasource.datasource_UIDs[curr_data_series_index]
# points for the current plot:
pts = np.column_stack([curr_data_series_dict['x'], curr_data_series_dict['y'], curr_data_series_dict['z']])
pts

In [ ]:
## Add the new filled plot item:
plot_args = ({'color_name':'white','line_width':0.5,'z_scaling_factor':1.0})
_test_fill_plt = gl.GLLinePlotItem(pos=points, color=line_color, width=plot_args.setdefault('line_width',0.5), antialias=True)
_test_fill_plt.scale(1.0, 1.0, plot_args.setdefault('z_scaling_factor',1.0)) # Scale the data_values_range to fit within the z_max_value. Shouldn't need to be adjusted so long as data doesn't change.            
# plt.scale(1.0, 1.0, self.data_z_scaling_factor) # Scale the data_values_range to fit within the z_max_value. Shouldn't need to be adjusted so long as data doesn't change.
active_curve_plotter_3d.ui.main_gl_widget.addItem(_test_fill_plt)

In [ ]:
active_curve_plotter_3d.plots.keys()

In [ ]:
active_curve_plotter_3d.params.render_epochs

In [ ]:
active_curve_plotter_3d.params.time_curves_datasource

In [ ]:
active_curve_plotter_3d.params.time_curves_enable_baseline_grid = True
active_curve_plotter_3d.params.time_curves_baseline_grid_alpha = 0.9
# add_3D_time_curves_baseline_grid_mesh

In [ ]:
active_curve_plotter_3d.add_3D_time_curves_baseline_grid_mesh()

In [ ]:
active_curve_plotter_3d.remove_3D_time_curves_baseline_grid_mesh()

In [ ]:
list(active_curve_plotter_3d.plots.keys())

In [ ]:
active_curve_plotter_3d.plots.time_curve_helpers

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.TimeCurves3D.Render3DTimeCurvesBaseGridMixin import BaseGrid3DTimeCurvesHelper, Render3DTimeCurvesBaseGridMixin

In [ ]:
BaseGrid3DTimeCurvesHelper.init_3D_time_curves_baseline_grid_mesh(active_curve_plotter_3d=active_curve_plotter_3d)

In [ ]:
BaseGrid3DTimeCurvesHelper.add_3D_time_curves_baseline_grid_mesh(active_curve_plotter_3d=active_curve_plotter_3d)

In [ ]:
BaseGrid3DTimeCurvesHelper.remove_3D_time_curves_baseline_grid_mesh(active_curve_plotter_3d=active_curve_plotter_3d)

In [ ]:
active_curve_plotter_3d.params.time_curves_main_alpha = 0.5
active_curve_plotter_3d.update_3D_time_curves()

In [ ]:
# Add default params if needed:
# active_curve_plotter_3d.params


In [ ]:
list(active_curve_plotter_3d.params.keys())

In [ ]:
active_curve_plotter_3d.params.time_curves_z_baseline

In [ ]:
active_curve_plotter_3d.plots

In [ ]:
'time_curve_helpers' not in active_curve_plotter_3d.plots

In [ ]:
'plots_grid_3dCurveBaselines_Grid' not in active_curve_plotter_3d.plots.time_curve_helpers

In [ ]:
time_curves_z_baseline = 5.0 

data_series_baseline
# z_map_fn = lambda v_main: v_main + 5.0 # returns the un-transformed primary value

5.0

In [ ]:
active_curve_plotter_3d.params.axes_walls_z_height = 15.0

In [ ]:
active_curve_plotter_3d._update_axes_plane_graphics()

In [ ]:
active_curve_plotter_3d.temporal_axis_length

In [ ]:
active_curve_plotter_3d.temporal_zoom_factor # 2.6666666666666665

In [ ]:
active_curve_plotter_3d.temporal_to_spatial(temporal_data=[1.0])

In [ ]:
line_color = pg.mkColor(plot_args.setdefault('color_name', 'white'))
line_color.setAlphaF(0.8)

# Pho Scratch Workspace

## Firing Rates 2022-07-06

### Old Placefield Firing Rate Tests:

In [ ]:
# debug_dump_object_member_shapes(active_one_step_decoder)
# computation_result.computed_data['pf2D_Decoder']
# active_one_step_decoder.time_window_edges

active_firing_rate_trends = curr_active_pipeline.computation_results[active_config_name].computed_data['firing_rate_trends']

active_rolling_window_times = active_firing_rate_trends['active_rolling_window_times']
mean_firing_rates = active_firing_rate_trends['mean_firing_rates']
moving_mean_firing_rates_df = active_firing_rate_trends['moving_mean_firing_rates_df']
moving_mean_firing_rates_df # 3969 rows x 43 columns

# mean_firing_rates
# pg.plot(mean_firing_rates)

np.shape(moving_mean_firing_rates_df) # (3969, 43)
good_only_moving_mean_firing_rates_df = moving_mean_firing_rates_df.dropna() # 3910 rows x 43 columns
good_only_moving_mean_firing_rates_df.T
err, win = _display_firing_rate_trends(good_only_moving_mean_firing_rates_df.T)
win.show()



In [ ]:
# active_computation_results[filter_name]
active_firing_rate_trends = active_computed_data['firing_rate_trends']
active_firing_rate_trends.keys() # ['active_rolling_window_times', 'mean_firing_rates', 'desired_window_length_seconds', 'desired_window_length_bins', 'active_firing_rates_df', 'moving_mean_firing_rates_df']

In [ ]:
moving_mean_firing_rates_df = active_firing_rate_trends['moving_mean_firing_rates_df']
moving_mean_firing_rates_df

In [ ]:
#### Compute the fireing rates in the windows corresponding to the sampling rate of the position bins, or better yet for the entire duration spent in a single position bin (splitting on bin-index changes

In [ ]:
# if 'binned_x' not in self._filtered_spikes_df:
#     self._filtered_spikes_df['binned_x'] = pd.cut(self._filtered_spikes_df['x'].to_numpy(), bins=self.xbin, include_lowest=True, labels=self.xbin_labels) # same shape as the input data 
    
# if 'binned_y' not in self._filtered_spikes_df:
#     self._filtered_spikes_df['binned_y'] = pd.cut(self._filtered_spikes_df['y'].to_numpy(), bins=self.ybin, include_lowest=True, labels=self.ybin_labels)

## Add Binned Position Columns to spikes_df:
def _build_df_discretized_binned_positions(active_df, active_computation_config, xbin_values=None, ybin_values=None, force_recompute=False, debug_print=False):
    """ Adds the 'binned_x' and 'binned_y' columns to the passed-in dataframe
    Requires that the passed in dataframe has at least the 'x' column (1D) and optionally the 'y' column.
    Works for both position_df and spikes_df
    
    Usage:
        active_df, xbin, ybin, bin_info = _build_df_discretized_binned_positions(active_pf_2D.filtered_spikes_df.copy(), active_computation_config, xbin_values=active_pf_2D.xbin, ybin_values=active_pf_2D.ybin, force_recompute=False, debug_print=True)
        active_df
    
    ## TODO: Move into perminant location and replace duplicated/specific implementations with this more general version.
        Known Reimplementations:
            neuropy.analyses.time_dependent_placefields.__init__(...)
            General\Decoder\decoder_result.py - build_position_df_discretized_binned_positions(...)
    """
    # bin the dataframe's x and y positions into bins, with binned_x and binned_y containing the index of the bin that the given position is contained within.
    if (xbin_values is None) or (ybin_values is None):
        # determine the correct bins to use from active_computation_config.grid_bin:
        if debug_print:
            print(f'active_grid_bin: {active_computation_config.grid_bin}')

        if 'y' in active_df.columns:
            # 2D case:
            # if (('binned_x' not in active_df.columns) or ('binned_y' not in active_df.columns)) and not force_recompute:
            xbin, ybin, bin_info = PfND._bin_pos_nD(active_df['x'].values, active_df['y'].values, bin_size=active_computation_config.grid_bin) # bin_size mode            
        else:
            # 1D case:
            # if ('binned_x' not in active_df.columns) and not force_recompute:
            xbin, ybin, bin_info = PfND._bin_pos_nD(active_df['x'].values, None, bin_size=active_computation_config.grid_bin) # bin_size mode
    else:
        # use the extant values passed in:
        if debug_print:
            print(f'using extant bins passed as arguments: xbin_values.shape: {xbin_values.shape}, ybin_values.shape: {ybin_values.shape}')
        xbin = xbin_values
        ybin = ybin_values
        bin_info = None

    if ('binned_x' not in active_df.columns) and not force_recompute:
        active_df['binned_x'] = pd.cut(active_df['x'].to_numpy(), bins=xbin, include_lowest=True, labels=np.arange(start=1, stop=len(xbin))) # same shape as the input data 
    if 'y' in active_df.columns:
        # Only do the y-variables in the 2D case.
        if ('binned_y' not in active_df.columns) and not force_recompute:
            active_df['binned_y'] = pd.cut(active_df['y'].to_numpy(), bins=ybin, include_lowest=True, labels=np.arange(start=1, stop=len(ybin))) 

    return active_df, xbin, ybin, bin_info


active_df, xbin, ybin, bin_info = _build_df_discretized_binned_positions(active_pf_2D.filtered_spikes_df.copy(), active_computation_config, xbin_values=active_pf_2D.xbin, ybin_values=active_pf_2D.ybin, force_recompute=False, debug_print=True)
active_df

In [ ]:
active_pf_2D.bin_info

In [ ]:
# sess.spikes_df # no 'binned_x' column
active_pf_2D.filtered_spikes_df

In [ ]:
#### Need to compute the best-known transition times between position bins for all spikes as not to truncate the starts/ends of the bins (which would happen if doing a single cell at a time)
# active_df.binned_x
active_df['_binned_x_transitions'] = active_df['binned_x'].diff()
active_df_filtered_binned_x_transitions_only = active_df[active_df['_binned_x_transitions'] != 0] # get only the transition points (for binned_x)
active_df_filtered_binned_x_transitions_only

active_df_filtered_binned_x_transitions_only = active_df_filtered_binned_x_transitions_only[['t_rel_seconds', 'binned_x']]
# Get durations spent in the bin:
active_df_filtered_binned_x_transitions_only['bin_occupied_duration'] = active_df_filtered_binned_x_transitions_only['t_rel_seconds'].diff() # How long the animal spent in this bin before transitioning (along its axis)
active_df_filtered_binned_x_transitions_only


In [ ]:
transition_start_spike_index = active_df_filtered_binned_x_transitions_only.index
transition_end_spike_index = transition_start_spike_index - 1 # the end of each occurs at the time of the previous one

In [ ]:
#### In general, for each neuron, get the spikes that occured during each bin_occupation_event 'i' (from t_rel_seconds[i] to (t_rel_seconds[i] + bin_occupied_duration[i])) and divide by the bin_occupied_duration[i]
# This will give the bin_occupation_event's spike_rate for each neuron which we can filter on (to find the neuron's that fire > 1Hz for this bin_occupation_event)

"""
Along a given axis (x or y), an animal occupies a single bin at each time given by (binned_x or binned_y). 
When the animal changes bins and a transition occurs along a given axis, it is said to terminate the occupation of the previous bin and start the occupation of the new bin.

These (start_t, bin_occupied_duration, binned_x) tuples denote a unique event, termed a `bin_occupation_event`.

Within a given bin_occupation_event each neuron has an instantaneous bin firing rate given by: (number of spikes it fired during the event) / bin_occupied_duration

"""

## Change to simplier non-time-dependent version:

In [ ]:
# Plot from the active_simpler_pf_densities_analysis
out_app, out_win, out_imageView = pyqtplot_plot_image(active_pf_2D_dt.xbin_labels, active_pf_2D_dt.ybin_labels, active_simpler_pf_densities_analysis.n_neurons_meeting_firing_critiera_by_position_bins_2D)

In [ ]:
# out = BinnedImageRenderingWindow(active_pf_2D_dt.curr_firing_maps_matrix, active_pf_2D_dt.xbin_labels, active_pf_2D_dt.ybin_labels)
out = BasicBinnedImageRenderingWindow(active_pf_2D_dt.curr_firing_maps_matrix, active_pf_2D_dt.xbin_labels, active_pf_2D_dt.ybin_labels)
# pyqtplot_plot_image(active_pf_2D_dt.xbin_labels, active_pf_2D_dt.ybin_labels, active_pf_2D_dt.curr_firing_maps_matrix)

In [ ]:
# Compute the updated counts:
current_spike_per_unit_per_bin_counts = active_df.value_counts(subset=['fragile_linear_neuron_IDX', 'binned_x', 'binned_y'], normalize=False, sort=False, ascending=True, dropna=True) # dropna=True
current_spike_per_unit_per_bin_counts # pandas.core.series.Series

In [ ]:
type(current_spike_per_unit_per_bin_counts) # pandas.core.series.Series
# current_spike_per_unit_per_bin_counts[0]

pd.DataFrame(current_spike_per_unit_per_bin_counts)

In [ ]:

debug_print = True
for (fragile_linear_neuron_IDX, xbin_label, ybin_label), count in current_spike_per_unit_per_bin_counts.iteritems():
    if debug_print:
        print(f'fragile_linear_neuron_IDX: {fragile_linear_neuron_IDX}, xbin_label: {xbin_label}, ybin_label: {ybin_label}, count: {count}')
    try:
        last_firing_maps_matrix[fragile_linear_neuron_IDX, xbin_label-1, ybin_label-1] += count
    except IndexError as e:
        print(f'e: {e}\n active_current_spike_df: {np.shape(active_current_spike_df)}, current_spike_per_unit_per_bin_counts: {np.shape(current_spike_per_unit_per_bin_counts)}\n last_firing_maps_matrix: {np.shape(last_firing_maps_matrix)}\n count: {count}')
        print(f' last_firing_maps_matrix[fragile_linear_neuron_IDX: {fragile_linear_neuron_IDX}, (xbin_label-1): {xbin_label-1}, (ybin_label-1): {ybin_label-1}] += count: {count}')
        raise e
        

In [ ]:
## Now want it arranged by-bin instead of by-neuron:


## LATER: Also need to collapse over all binned_y for each binned_x (marginalize) - I think

## Get 2D Heatmaps of Velocity and Occupancy working:

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, add_bin_ticks, build_binned_imageItem

In [ ]:
out = BasicBinnedImageRenderingWindow(active_eloy_analysis.avg_2D_speed_per_pos, active_pf_2D.xbin, active_pf_2D.ybin, name='avg_velocity', title="Avg Velocity per Pos (X, Y)", variable_label='Avg Velocity')
out.add_data(row=2, col=0, matrix=active_eloy_analysis.pf_overlapDensity_2D, xbins=active_pf_2D.xbin, ybins=active_pf_2D.ybin, name='pf_overlapDensity', title='pf overlapDensity metric', variable_label='pf overlapDensity')
out.add_data(row=3, col=0, matrix=active_pf_2D.ratemap.occupancy, xbins=active_pf_2D.xbin, ybins=active_pf_2D.ybin, name='occupancy_seconds', title='Seconds Occupancy', variable_label='seconds')
out.add_data(row=4, col=0, matrix=active_simpler_pf_densities_analysis.n_neurons_meeting_firing_critiera_by_position_bins_2D, xbins=active_pf_2D.xbin, ybins=active_pf_2D.ybin, name='n_neurons_meeting_firing_critiera_by_position_bins_2D', title='# neurons > 1Hz per Pos (X, Y)', variable_label='# neurons')

## Test getting prominences with peak_prominence2d

In [ ]:
curr_display_function_name = '_display_pf_peak_prominence2d_default_quadrant_plots'
out_figs, out_axes, out_idxs = curr_active_pipeline.display(curr_display_function_name, active_config_name) 

In [ ]:
curr_display_function_name = 'plot_Prominence'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    for an_idx, a_fig in zip(active_peak_prominence_2d_results.neuron_extended_ids, out_figs):
        a_fig.suptitle(f'neuron: {an_idx.id}', fontsize=16)
        pdf.savefig(a_fig)

In [ ]:
curr_display_function_name = '_display_pf_peak_prominence2d_plots'
figure, ax = curr_active_pipeline.display(curr_display_function_name, active_config_name, neuron_id=2) 

# Quantitatively Analyzing the prominence computation results

In [ ]:
def min_mean_max_sum(M, print_result=True):
    out = (np.nanmin(M), np.nanmean(M), np.nanmax(M), np.nansum(M))
    if print_result:
        print(f'min: {out[0]}, mean: {out[1]}, max: {out[2]}, sum: {out[3]}')
    return out

In [ ]:
valid_neuron_id = 2
# valid_neuron_id = kwargs.get('neuron_id', 2)

In [8]:
## For a single neuron:
## Care about:
# - Pf Num of Peaks: expresses whether a cell responds selectively to a single location, is bi-modal, tri-modal, etc.
# - Peak Locations: (to determine the number of peaks per bin or region)
# - Peak Prominences/Relative heights
#   Pf Sizes: defined by our bounding boxes
#   Stability??

# QuantPeakResult
list(active_peak_prominence_2d_results.keys())

['xx', 'yy', 'neuron_extended_ids', 'results', 'flat_peaks_df']

In [ ]:
pf_summits_analysis_df = active_peak_prominence_2d_results.flat_peaks_df
pf_summits_analysis_df

In [9]:
def build_filtered_summits_analysis_results(xbin, ybin, xbin_labels, ybin_labels, active_peak_prominence_2d_results, slice_level_multiplier=0.5, minimum_included_peak_height=1.0, debug_print = False):
    """ builds the filtered summits analysis results dataframe and flat counts matrix 
    
    Usage:
        filtered_summits_analysis_df, pf_peak_counts_map = build_filtered_summits_analysis_results(active_pf_2D.xbin, active_pf_2D.ybin, active_pf_2D.xbin_labels, active_pf_2D.ybin_labels,
                                                                                           active_peak_prominence_2d_results, slice_level_multiplier=0.5, minimum_included_peak_height=1.0, debug_print = False)
                                                                                           
    """
    filtered_summits_analysis_df = pf_summits_analysis_df[pf_summits_analysis_df['peak_height'] >= minimum_included_peak_height].copy() # filter for peaks greater than 1.0Hz
    # filtered_summits_analysis_df
    ## Find which position bin each peak falls in:
    if 'peak_center_binned_x' not in filtered_summits_analysis_df:
        filtered_summits_analysis_df['peak_center_binned_x'] = pd.cut(filtered_summits_analysis_df['peak_center_x'].to_numpy(), bins=xbin, include_lowest=True, labels=xbin_labels) # same shape as the input data
    if 'peak_center_binned_y' not in filtered_summits_analysis_df:
        filtered_summits_analysis_df['peak_center_binned_y'] = pd.cut(filtered_summits_analysis_df['peak_center_y'].to_numpy(), bins=ybin, include_lowest=True, labels=ybin_labels)
    # filtered_summits_analysis_df
    ## IMPORTANT: Filter by only one of the slice_levels before continuing, otherwise you're double-counting:
    filtered_summits_analysis_df = filtered_summits_analysis_df[filtered_summits_analysis_df['slice_level_multiplier'] == slice_level_multiplier].copy()

    ## Build outputs:
    n_xbins = len(xbin) - 1 # the -1 is to get the counts for the centers only
    n_ybins = len(ybin) - 1 # the -1 is to get the counts for the centers only
    pf_peak_counts_map = np.zeros((n_xbins, n_ybins), dtype=int) # create an initially zero matrix

    current_bin_counts = filtered_summits_analysis_df.value_counts(subset=['peak_center_binned_x', 'peak_center_binned_y'], normalize=False, sort=False, ascending=True, dropna=True) # current_bin_counts: a series with a MultiIndex index for each bin that has nonzero counts
    if debug_print:
        print(f'np.shape(current_bin_counts): {np.shape(current_bin_counts)}') # (247,)
    for (xbin_label, ybin_label), count in current_bin_counts.iteritems():
        if debug_print:
            print(f'xbin_label: {xbin_label}, ybin_label: {ybin_label}, count: {count}')
        try:
            pf_peak_counts_map[xbin_label-1, ybin_label-1] += count
        except IndexError as e:
            print(f'e: {e}\n filtered_summits_analysis_df: {np.shape(filtered_summits_analysis_df)}, current_bin_counts: {np.shape(current_bin_counts)}\n pf_peak_counts_map: {np.shape(pf_peak_counts_map)}')
            raise e

    return filtered_summits_analysis_df, pf_peak_counts_map
    
filtered_summits_analysis_df, pf_peak_counts_map = build_filtered_summits_analysis_results(active_pf_2D.xbin, active_pf_2D.ybin, active_pf_2D.xbin_labels, active_pf_2D.ybin_labels,
                                                                                           active_peak_prominence_2d_results, slice_level_multiplier=0.5, minimum_included_peak_height=1.0, debug_print = False)

In [10]:
# Plot from the active_simpler_pf_densities_analysis
out_app, out_win, out_imageView = pyqtplot_plot_image(active_pf_2D.xbin_labels, active_pf_2D.ybin_labels, pf_peak_counts_map)

In [18]:
from scipy.ndimage.filters import uniform_filter, gaussian_filter
pf_peak_counts_map_blurred = uniform_filter(pf_peak_counts_map.astype('float'), size=3, mode='constant')
# pf_peak_counts_map_blurred

out_app, out_win, out_imageView = pyqtplot_plot_image(active_pf_2D.xbin_labels, active_pf_2D.ybin_labels, pf_peak_counts_map_blurred)

## Testing Variable Viewers

In [ ]:
from pyphocorehelpers.gui.tkinter.tk_tree_view import tk_tree_view

In [ ]:
def visualise_dict(d, lvl=0):
    # go through the dictionary alphabetically 
    for k in sorted(d):
        # print the table header if we're at the beginning
        if lvl == 0 and k == sorted(d)[0]:
            print('{:<25} {:<15} {:<10}'.format('KEY','LEVEL','TYPE'))
            print('-'*79)

        indent = '  '*lvl # indent the table to visualise hierarchy
        t = str(type(d[k]))

        # print details of each entry
        print("{:<25} {:<15} {:<10}".format(indent+str(k),lvl,t))

        # if the entry is a dictionary
        if type(d[k])==dict:
            # visualise THAT dictionary with +1 indent
            visualise_dict(d[k],lvl+1)

In [ ]:
from pyphocorehelpers.print_helpers import print_value_overview_only, print_keys_if_possible, pprint, debug_dump_object_member_shapes

In [ ]:
# tk_tree_view(peaks[1])
# tk_tree_view(peaks)

# tk_tree_view(ipcDataExplorer.plots.to_dict())
visualise_dict(ipcDataExplorer.plots)

In [ ]:
str(type(tuningCurvePlotData[2]['pdata_currActiveNeuronTuningCurve']))

# pyvista.core.pointset.StructuredGrid

In [ ]:
# print_keys_if_possible('plots_data', ipcDataExplorer.plots_data) # RenderPlotsData
tuningCurvePlotData = ipcDataExplorer.plots_data['tuningCurvePlotData']
# - tuningCurvePlotData: <class 'indexed.IndexedOrderedDict'>
# 	- 2: <class 'dict'>
# 		- curr_active_neuron_ID: <class 'numpy.int32'>
# 		- curr_active_neuron_pf_identifier: <class 'str'>
# 		- curr_active_neuron_tuning_Curve: <class 'numpy.ndarray'> - (29, 64)
# 		- pdata_currActiveNeuronTuningCurve: <class 'pyvista.core.pointset.StructuredGrid'> - OMITTED TYPE WITH NO SHAPE
# 		- pdata_currActiveNeuronTuningCurve_Points: <class 'pyvista.core.pointset.UnstructuredGrid'> - OMITTED TYPE WITH NO SHAPE
# 		- lut: <class 'vtkmodules.vtkCommonCore.vtkLookupTable'>
        
# 'occupancyPlotData'
# tuningCurvePlotData.shape
print_keys_if_possible('tuningCurvePlotData', tuningCurvePlotData) # RenderPlotsData

In [ ]:
print_keys_if_possible('ipcDataExplorer.plots', ipcDataExplorer.plots, additional_excluded_item_classes=["<class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'>", "<class 'vtkmodules.vtkRenderingAnnotation.vtkLegendBoxActor'>"]) # RenderPlots
# pcDataExplorer.plots: <class 'pyphocorehelpers.DataStructure.general_parameter_containers.RenderPlots'>
# 	- name: <class 'str'>
# 	- occupancyPlotActor: <class 'NoneType'>
# 	- maze_bg: <class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'> - OMITTED TYPE WITH NO SHAPE
# 	- tuningCurvePlotActors: <class 'indexed.IndexedOrderedDict'>
# 		- 2: <class 'pyphocorehelpers.gui.PyVista.CascadingDynamicPlotsList.CascadingDynamicPlotsList'>
# 			- main: <class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'> - OMITTED TYPE WITH NO SHAPE
# 			- points: <class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'> - OMITTED TYPE WITH NO SHAPE
#           ...
#         - 64: <class 'pyphocorehelpers.gui.PyVista.CascadingDynamicPlotsList.CascadingDynamicPlotsList'>
# 			- main: <class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'> - OMITTED TYPE WITH NO SHAPE
# 			- points: <class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'> - OMITTED TYPE WITH NO SHAPE
# 	- tuningCurvePlotLegendActor: <class 'vtkmodules.vtkRenderingAnnotation.vtkLegendBoxActor'> - OMITTED TYPE WITH NO SHAPE
# 	- spikes_pf_active: <class 'vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor'> - OMITTED TYPE WITH NO SHAPE
        
active_tuning_curve_plot_actors = ipcDataExplorer.plots['tuningCurvePlotActors']


In [ ]:
debug_dump_object_member_shapes(tuningCurvePlotData)

In [ ]:
visualise_dict(ipcDataExplorer.plots, lvl=1)

In [ ]:
active_neuron_id = 2
tuning_curve_is_visible = ipcDataExplorer.plots['tuningCurvePlotActors'][active_neuron_id].main.GetVisibility()
tuning_curve_is_visible

In [ ]:
# ipcDataExplorer.p.remove_actor(contours_mesh)

## Vedo display of contours testing

In [ ]:
plt = plot(active_peak_prominence_2d_results.xx, active_peak_prominence_2d_results.yy, slab, c='summer') # use a colormap
show(plt, viewup='z').close()

In [ ]:
'''Draw a z = f(x,y) surface specified as
a string or as a reference to an external function.
Red points indicate where the function does not exist!'''
from vedo import dataurl, sin, cos, log, show, Text2D
from vedo.pyplot import plot

doc = Text2D(__doc__, pos='bottom-left', c='darkgreen', font='Quikhand')

############################################################### REAL
def f(x, y):
    return sin(2*x*y) * cos(3*y)/2
f1 = plot(f, c='summer') # use a colormap

# red dots are shown where the function does not exist (y>x):
def f(x, y):
    return sin(3*x) * log(x-y)/3
f2 = plot(f, texture=dataurl+'textures/paper3.jpg')

# specify x and y ranges and z vertical limits:
def f(x, y):
    return log(x**2+y**2-1)
f3 = plot(
    f,
    xlim=[-2,2],
    ylim=[-1,8],
    zlim=[-1,None],
    texture=dataurl+'textures/paper1.jpg',
)

show([(f1, 'y = sin(2*x*y) * cos(3*y) /2', doc),
      (f2, 'y = sin(3*x)*log(x-y)/3'),
      (f3, 'y = log(x**2+y**2-1)'),
     ], N=3, sharecam=False,
).close()

############################################################## COMPLEX
comment = """Vertical axis shows the real part of complex z:
    z = sin(log(x\doty))
Color map the value of the imaginary part
(green=positive, purple=negative)"""

plt = plot(lambda x,y: sin(log(x*y))/25, mode='complex')

show(plt, comment, viewup='z').close()

In [ ]:
isol = mesh0.isolines(n=10).color('w')
isob = mesh0.isobands(n=5).addScalarBar(title="H=Elevation")

# make a copy and interpolate the Scalars from points to cells
mesh1 = mesh0.clone(deep=False).mapPointsToCells()
printc('Mesh cell arrays :', mesh1.celldata.keys())

gvecs = mesh1.gradient(on='cells')
cc = mesh1.cellCenters()
ars = Arrows(cc, cc + gvecs*0.01, c='bone_r').lighting('off')
ars.addScalarBar3D(title='|\nablaH|~\dot~0.01 [arb.units]')

# colormap the gradient magnitude directly on the mesh
mesh2 = mesh1.clone(deep=False).lw(0.1).cmap('jet', mag(gvecs), on='cells')
mesh2.addScalarBar3D(title='|\nablaH| [arb.units]')

plt = Plotter(N=4, size=(1200,900), axes=11)
plt.at(0).show(mesh0, isol, __doc__)
plt.at(1).show(isob)
plt.at(2).show(mesh1, isol, ars, "Arrows=\nablaH")
plt.at(3).show(mesh2, "Color=|\nablaH|")
plt.interactive().close()